# 第3节：校准

## 表 3.1--模型核心参数校准

In [ ]:
% =========================================================================
% == SCRIPT: Tab3_1.m
% == 版本: [v2.7 - 精简换行版]
% ==
% == 核心修改:
% ==   - [!!! 格式优化 !!!] 采纳您的建议，在 \midrule, \toprule,
% ==     \bottomrule 和 \multicolumn 这些本身就定义了行结构的命令后，
% ==     不再输出多余的换行符 "\\"。
% =========================================================================
clear; clc;
addpath(pwd);
fprintf('--- 启动 LaTeX 参数表直接生成脚本 (v2.7) ---\n');

%% --- 1. 加载数据并定义输出文件名 ---
data_filename = 'SS/data_for_het_transition_nopps.mat';
output_filename = 'tex/tab/Tab3.1.tex';

try
    load(data_filename, 'data_for_transition');
    cS = data_for_transition.cS;
    fprintf('   ✅ 已从 "%s" 加载参数结构体 cS。\n', data_filename);
catch ME
    error('无法加载转轨数据文件 "%s"。请先成功运行 main_run_ss.m。\n错误信息: %s', data_filename, ME.message);
end

%% --- 2. 构建参数数据 (Cell Array) ---
% 格式: { '命令或参数名', '符号', '数值', '来源', is_full_line_command }
% 最后一列是布尔值，用于标记是否为独占一行的命令
params_data = {
    '\midrule[0.75pt]', '', '', '', true; 
    '\multicolumn{4}{l}{\textbf{A. 偏好与生命周期参数}}\\', '', '', '', true;
    '相对风险规避系数', '$\sigma$', sprintf('%.1f', cS.sigma), '标准宏观文献', false;
    '主观贴现因子 (年化)', '$\beta_{annual}$', sprintf('%.3f', cS.beta^(1/cS.time_Step)), '校准目标', false;
    '存活率', '$s_a$', '时变', 'UN WPP (2024)', false;
    '出生率', '', '时变', 'UN WPP (2024)', false;
    '\midrule', '', '', '', true; 
    '\multicolumn{4}{l}{\textbf{B. 生产技术参数}}\\', '', '', '', true;
    '长期技术进步率 (年化)', '$g_{A,ss}$', sprintf('%.1f%%', cS.g_A_ss * 100), '校准目标', false;
    '私人资本产出弹性', '$\alpha$', sprintf('%.3f', cS.alpha), '校准目标', false;
    '公共资本产出弹性', '$\gamma$', sprintf('%.3f', cS.gamma), '校准目标', false;
    '资本折旧率 (年化)', '$\delta_g$', sprintf('%.1f%%', (1-(1-cS.ddk_g)^(1/cS.time_Step))*100), '校准目标', false;
    '\midrule', '', '', '', true;
    '\multicolumn{4}{l}{\textbf{C. 财政与PAYG养老金}}\\', '', '', '', true;
    '资本所得税率', '$\tau_k$', sprintf('%.1f%%', cS.tau_k * 100), '中国现行税制', false;
    '劳动所得税率', '$\tau_l$', sprintf('%.1f%%', cS.tau_l * 100), '中国现行税制', false;
    '消费税率', '$\tau_c$', sprintf('%.1f%%', cS.tau_c * 100), '中国现行税制', false;
    '政府消费 / GDP', '$G_c/Y$', sprintf('%.1f%%', cS.G_c_to_Y_ratio_ss * 100), '财政数据', false;
    '公共投资 / GDP', '$I_g/Y$', sprintf('%.1f%%', cS.I_g_to_Y_ratio_ss * 100), '财政数据', false;
    '城镇职工PAYG缴费率', '$\theta_{h,t} (h=1,2)$', sprintf('%.1f%%', 0.2 * 100), '内生以平衡PAYG', false;
    '城乡居民PAYG缴费率', '$\theta_{h,t} (h=3,4)$', sprintf('%.1f%%', 0.03 * 100), '内生以平衡PAYG', false;
    '\midrule', '', '', '', true;
    '\multicolumn{4}{l}{\textbf{D. 个人养老金 (PPS) 制度}}\\', '', '', '', true;
    'PPS最大缴费率 (占劳动收入)', '$pps_{max}$', sprintf('%.1f%%', cS.pps_fixed * 100), '中国个人养老金政策', false;
    'PPS提取率 (占PPS资产)', '', sprintf('%.1f%%', cS.pps_withdrawal_rate * 100), '模型简化假设', false;
    'PPS提取税率', '$\tau_{pps}$', sprintf('%.1f%%', cS.pps_tax_rate_withdrawal * 100), '中国个人养老金政策', false;
};
fprintf('   ✅ 已构建参数数据。\n');

%% --- 3. 准备输出目录并打开文件 ---
[output_dir, ~, ~] = fileparts(output_filename);
if ~exist(output_dir, 'dir')
    mkdir(output_dir);
    fprintf('   已创建新目录: "%s"\n', output_dir);
end

fileID = fopen(output_filename, 'w', 'n', 'UTF-8');
if fileID == -1, error('无法创建或打开最终输出文件: %s', output_filename); end
fprintf('   正在将完整的LaTeX表格代码写入 "%s"...\n', output_filename);

% --- 写入表格外层框架 ---
fprintf(fileID, '\\begin{table}[h!]\n');
fprintf(fileID, '\\centering\n');
fprintf(fileID, '\\caption{模型核心参数校准}\n');
fprintf(fileID, '\\label{tab:param_calibration}\n');
fprintf(fileID, '\\begin{threeparttable}\n');
fprintf(fileID, '\\begin{tabular}{lccc}\n');
fprintf(fileID, '\\toprule\n');
fprintf(fileID, '参数名 & 参数符号 & 参数值 & 参数来源 \\\\\n');

% --- 循环写入表格主体内容 ---
for i = 1:size(params_data, 1)
    % --- 统一进行转义处理 ---
    col1 = strrep(params_data{i, 1}, '%', '\%');
    col2 = strrep(params_data{i, 2}, '%', '\%');
    col3 = strrep(params_data{i, 3}, '%', '\%');
    col4 = strrep(params_data{i, 4}, '%', '\%');
    is_command = params_data{i, 5};
    
    if is_command
        % 如果是整行命令，直接打印，不加换行符 "\\"
        fprintf(fileID, '%s\n', col1);
    else
        % 如果是普通的数据行，用 "&" 连接并换行
        fprintf(fileID, '%s & %s & %s & %s \\\\\n', col1, col2, col3, col4);
    end
end

% --- 写入表格结尾和注释 ---
fprintf(fileID, '\\bottomrule\n');
fprintf(fileID, '\\end{tabular}\n'); 
fprintf(fileID, '\\begin{tablenotes}[para]\n');
fprintf(fileID, '  \\item 注： 主观贴现因子、折旧率和人口增长率已根据模型周期长度 (%d年) 进行年化处理以便解读。\n', cS.time_Step);
fprintf(fileID, '  PAYG缴费率($\\theta_t$)和存活率($\\phi_a$)为时变路径，详情见正文。\n');
fprintf(fileID, '\\end{tablenotes}\n');
fprintf(fileID, '\\end{threeparttable}\n');
fprintf(fileID, '\\end{table}');

% --- 关闭文件 ---
fclose(fileID);

fprintf('   ✅ 成功生成语法正确的LaTeX表格文件: %s\n', output_filename);
fprintf('\n--- 脚本执行完毕 ---\n');

--- 启动 LaTeX 参数表生成脚本 ---
   ✅ 已从 "SS/data_for_transition.mat" 加载参数结构体 cS。
   ✅ 已根据模型设定构建参数表。
   ✅ 已调用 table2latex 生成临时文件 "temp_table_for_script.tex"。
   正在将表格内容写入最终框架并保存至 "Tab3.1.txt"...
   ✅ 成功生成最终的LaTeX表格文件: Tab3.1.txt

--- 脚本执行完毕 ---


## 图4.0 我国2023-2300年外生人口动态与结构演变

In [ ]:
% =========================================================================
% == SCRIPT: create_population_visuals.m
% == 版本: [v1.8 - 外生出生率精确重现版]
% ==
% == 目的:
% ==   1. 从指定结果文件中加载 cS 结构体。
% ==   2. 新增逻辑，读取原始UN数据并重现与 population.m 完全一致的外生出生率路径。
% ==   3. 为3D图表设置独立的年份上限，以展示长期结构演变。
% ==   4. 将最终图表保存为 tex/fig/fig4.0_population.png。
% ==
% == 前置条件:
% ==   - 已成功运行 main_run_trans.m 并生成以下文件:
% ==     - TRANS/TPI_results_het_nopps_raw.mat
% ==   - MATLAB路径下存在人口数据文件:
% ==     - data/人口/UN_PPP2024_CBR_birthper1000_China.xlsx
% =========================================================================

clear; close all; clc;
addpath(pwd);
fprintf('=== 人口路径详尽可视化脚本 (v1.8) ===\n\n');

%% --- 1. 用户设定与数据加载 ---
fprintf('--- 1. 用户设定与数据加载 ---\n');

% --- 输入文件 ---
input_file = 'TRANS/TPI_results_het_nopps_raw.mat';

% --- 输出文件与可视化年份上限 ---
output_dir = 'tex/fig/';
output_filename = fullfile(output_dir, 'fig4.0_population.png');
year_max = 2300; % <--- 设置图1和图3的最大年份
year_max_3d = 2150; % <--- 为3D图单独设置最大年份

% --- 加载数据 ---
if ~exist(input_file, 'file')
    error('找不到输入文件: %s，请确认main_run_trans.m已成功运行。', input_file);
end
fprintf('   正在从 "%s" 加载 cS 结构体...\n', input_file);
loaded_data = load(input_file, 'cS');
cS = loaded_data.cS;
fprintf('   ✅ cS 结构体加载成功。\n');

%% --- 2. 数据准备与指标计算 ---
fprintf('\n--- 2. 数据准备与指标计算 ---\n');

% --- 提取完整核心数据 ---
Z_path_raw_full = cS.Z_path_raw; 
T_sim = cS.T_sim;
time_step = cS.time_Step;
time_axis_full = cS.start_year : time_step : cS.end_year;
if length(time_axis_full) > T_sim, time_axis_full = time_axis_full(1:T_sim); end
annual_years_vec_full = cS.start_year:cS.end_year;

% --- A. 为图1和图3准备数据 (使用 year_max) ---
display_idx = time_axis_full <= year_max;
time_axis = time_axis_full(display_idx);
Z_path_raw_main = Z_path_raw_full(:, display_idx);
T_display_main = length(time_axis);

% 计算20岁以上总人口及其增长率
total_pop_path = sum(Z_path_raw_main, 1);
if T_display_main > 1
    g_n_period = total_pop_path(2:end) ./ total_pop_path(1:end-1) - 1;
    g_n_annual = (1 + g_n_period).^(1/time_step) - 1;
    time_axis_growth = time_axis(1:end-1) + time_step/2;
else
    g_n_annual = [];
    time_axis_growth = [];
end

% --- [核心修改] 重现 population.generate_Z_path 中的外生出生率路径 ---
fprintf('   正在重现外生年化出生率路径以确保一致性...\n');
try
    cbr_data = readtable('data/人口/UN_PPP2024_CBR_birthper1000_China.xlsx');
catch ME
    error('无法加载出生率数据文件: %s', ME.message);
end
var_names_cbr = cbr_data.Properties.VariableNames;
year_cols_indices_cbr = find(startsWith(var_names_cbr, 'y'));
un_years = str2double(cellfun(@(x) x(2:end), var_names_cbr(year_cols_indices_cbr), 'UniformOutput', false));
un_cbr = table2array(cbr_data(1, year_cols_indices_cbr)) / 1000;

last_data_year = 2100;
birth_rate_path_annual_full = zeros(size(annual_years_vec_full));
data_mask = annual_years_vec_full <= last_data_year;
birth_rate_path_annual_full(data_mask) = interp1(un_years, un_cbr, annual_years_vec_full(data_mask), 'pchip', 'extrap');
last_data_rate = birth_rate_path_annual_full(find(data_mask, 1, 'last'));
ss_mask = annual_years_vec_full > last_data_year;
birth_rate_path_annual_full(ss_mask) = last_data_rate;
birth_rate_path_annual_full = birth_rate_path_annual_full * 1000; % 转换为每千人单位

% 根据 year_max 筛选最终用于绘图的出生率数据
annual_display_idx = annual_years_vec_full <= year_max;
birth_rate_for_plot = birth_rate_path_annual_full(annual_display_idx);
time_axis_annual_br = annual_years_vec_full(annual_display_idx);
fprintf('   ✅ 年化出生率路径重现成功。\n');

% 使用固定的60岁基准，计算用于可视化的平滑老龄化指标
aR_fixed_60 = ceil((60 - cS.age1_orig) / cS.time_Step);
working_pop_for_plot = sum(Z_path_raw_main(1:aR_fixed_60, :), 1);
retiree_pop_for_plot = sum(Z_path_raw_main((aR_fixed_60+1):end, :), 1);
dependency_ratio_plot = (retiree_pop_for_plot ./ working_pop_for_plot) * 100;
retiree_share_plot = (retiree_pop_for_plot ./ total_pop_path) * 100;

% --- B. 为3D图单独准备数据 (使用 year_max_3d) ---
display_idx_3d = time_axis_full <= year_max_3d;
time_axis_3d = time_axis_full(display_idx_3d);
Z_path_raw_3d = Z_path_raw_full(:, display_idx_3d);
total_pop_path_3d = sum(Z_path_raw_3d, 1);

pop_share_path_3d = Z_path_raw_3d ./ total_pop_path_3d * 100; 
sample_interval = 5; 
sampled_indices_3d = 1:sample_interval:length(time_axis_3d);
sampled_years_3d = time_axis_3d(sampled_indices_3d);
sampled_Z_data_relative_3d = pop_share_path_3d(:, sampled_indices_3d);

age_group_labels = cell(cS.aD_new, 1);
for a = 1:cS.aD_new
    age_start = cS.age1_orig + (a-1)*time_step;
    age_end = age_start + time_step - 1;
    age_group_labels{a} = sprintf('%d-%d', age_start, age_end);
end

fprintf('   ✅ 核心人口指标计算完毕。\n');

%% --- 3. [核心] 生成可视化图表 ---
fprintf('\n--- 3. 生成可视化图表 ---\n');

% --- 初始化图窗, 采用 1x3 布局和指定尺寸 ---
fig = figure('Name', '人口动态与结构演变', 'Position', [200 504 1077 258]);
color_dark_gray = [0.3 0.3 0.3];
color_medium_gray = [0.5 0.5 0.5];
color_light_gray = [0.6 0.6 0.6];

% --- 图 1: 20岁以上人口总量与增长率 ---
subplot(1, 3, 1);
yyaxis left;
p1 = plot(time_axis, total_pop_path / 1e5, '-s', 'Color', color_dark_gray, 'LineWidth', 1.5, 'MarkerFaceColor', color_dark_gray, 'MarkerSize', 3);
ylabel('人口 (亿人)', 'Color', color_dark_gray);
ax1 = gca;
ax1.YAxis(1).Color = color_dark_gray;

yyaxis right;
p2 = plot(time_axis_growth, g_n_annual * 100, '-.', 'Color', color_medium_gray, 'LineWidth', 1.5, 'MarkerFaceColor', color_medium_gray, 'MarkerSize', 3);
yline(0, '--', 'Color', color_dark_gray);
ylabel('年化增长率 (%)', 'Color', color_medium_gray);
ax1.YAxis(2).Color = color_medium_gray;
ylim([-5,2])

title('20岁以上人口总量与增长率');
legend([p1, p2], {'总人口', '年增长率'}, 'Location', 'best');
grid on; box on;
xlim([time_axis(1), time_axis(end)]);
xticks(2020:20:year_max);
xtickangle(45);

% --- 图 2: 人口相对结构演变 (3D柱状图) ---
subplot(1, 3, 2);
b = bar3(sampled_Z_data_relative_3d');
title('人口相对结构演变');
xlabel('年龄组');
ylabel('年份');
zlabel('人口占比 (%)');
set(gca, 'XTick', 1:3:cS.aD_new, 'XTickLabel', age_group_labels(1:3:cS.aD_new));
set(gca, 'YTickLabel', sampled_years_3d);
xtickangle(45);
ytickangle(-45);
colormap(flipud(gray));
shading interp;
view(-1.179284176186033e+02,17.605013023005583);

% --- 图 3: 老龄化趋势与出生率 ---
subplot(1, 3, 3);
yyaxis left;
p3 = plot(time_axis, dependency_ratio_plot, '-o', 'Color', color_dark_gray, 'LineWidth', 1.5, 'MarkerFaceColor', color_dark_gray, 'MarkerSize', 3);
hold on;
p4 = plot(time_axis, retiree_share_plot, '--^', 'Color', color_medium_gray, 'LineWidth', 1.5, 'MarkerFaceColor', color_medium_gray, 'MarkerSize', 3);
ylabel('比率 (%)');
ax3 = gca;
ax3.YAxis(1).Color = color_dark_gray;

yyaxis right;
p5 = plot(time_axis_annual_br, birth_rate_for_plot, '-.', 'Color', color_light_gray, 'LineWidth', 1.5);
ylabel('年化出生率 (每千人)', 'Color', color_medium_gray);
ax3.YAxis(2).Color = color_medium_gray;
ylim([1,5.2]);

title('老龄化趋势与出生率');
legend([p3, p4, p5], {'老年抚养比', '退休人口占比', '年化出生率'}, 'Location', 'best');
grid on; box on;
xlim([time_axis(1), time_axis(end)]);
xticks(2020:20:year_max);
xtickangle(45);

fprintf('   ✅ 可视化图表生成完毕。\n');

%% --- 4. 保存图像 ---
fprintf('\n--- 4. 保存图像 ---\n');

if ~exist(output_dir, 'dir')
    mkdir(output_dir);
    fprintf('   已创建输出目录: %s\n', output_dir);
end

try
    exportgraphics(fig, output_filename, 'Resolution', 600);
    fprintf('   ✅ 图像已成功保存至: %s\n', output_filename);
catch ME
    fprintf('   ❌ 图像保存失败！错误信息: %s\n', ME.message);
end

fprintf('\n--- 脚本执行完毕 ---\n');

# 第4节：基准情形

## 表 4.0--稳态宏观经济结果比较

In [ ]:
% =========================================================================
% == SCRIPT: ss_comparison.m
% == 版本: [v2.2 - PAYG报告内容修正版]
% ==
% == 目的:
% ==   1. 加载单个转轨路径模拟的结果文件。
% ==   2. 提取并比较该路径的"初期稳态(ss0)"与"终期稳态(ssF)"。
% ==   3. 报告PAYG体系的规则化福利、总缴费和均衡调整因子(adj)。
% ==   4. 自动生成一个可直接用于论文的LaTeX表格 (Tab4.0_ss_comparison.tex)。
% ==
% == 前置条件:
% ==   - 已成功运行 main_run_trans.m 并生成以下文件:
% ==     - TRANS/TPI_results_het_nopps.mat
% =========================================================================

clear; close all; clc;
addpath(pwd);
fprintf('=== 转轨路径稳态比较与LaTeX表格生成脚本 (v2.2) ===\n\n');

%% --- 1. 加载数据 ---
fprintf('--- 1. 加载转轨路径的稳态数据 ---\n');

% --- 文件路径定义 ---
file_path = 'TRANS/TPI_results_het_nopps.mat';
output_tex_filename = 'tex/tab/Tab4.0_ss_comparison.tex';

% --- 加载数据 ---
try
    loaded_data = load(file_path, 'results', 'cS', 'ss0', 'ssF');
    results = loaded_data.results;
    ss0 = loaded_data.ss0;
    ssF = loaded_data.ssF;
    cS = loaded_data.cS;
    paramS_for_report = results.ss_data.paramSF;

    % --- 为核算报告重构稳态人口分布 (Z_ss_norm) 和总人口 ---
    Z0_total_abs = cS.Z_path_raw(:, 1);
    Z_ss_norm_0 = (Z0_total_abs / sum(Z0_total_abs)) * cS.type_weights';
    mass_total_0 = sum(Z_ss_norm_0, 'all');
    
    ZF_total_abs = cS.Z_path_raw(:, end);
    Z_ss_norm_F = (ZF_total_abs / sum(ZF_total_abs)) * cS.type_weights';
    mass_total_F = sum(Z_ss_norm_F, 'all');

    fprintf('   ✅ 已从 "%s" 加载转轨数据、初始稳态(ss0)和终期稳态(ssF)。\n', file_path);
catch ME
    error('加载转轨数据失败: %s', ME.message);
end

fprintf('   数据加载完成。\n');

%% --- 2. 生成国民经济核算报告 ---
fprintf('\n--- 2. 生成国民经济核算报告 ---\n');

% --- 为ss0和ssF分别生成报告结构体 (关闭命令行打印) ---
report_ss0 = SS.display_national_accounts(ss0, cS, paramS_for_report, Z_ss_norm_0, '', false, false);
report_ssF = SS.display_national_accounts(ssF, cS, paramS_for_report, Z_ss_norm_F, '', false, true);

% --- [核心修改] 手动计算规则化总福利，并添加到报告结构体中 ---
total_formula_benefit_ss0 = 0;
aR_idx_0 = cS.aR_new_path(1);
for h_idx = 1:cS.nTypes
    mass_retirees_h = sum(Z_ss_norm_0((aR_idx_0+1):end, h_idx));
    total_formula_benefit_ss0 = total_formula_benefit_ss0 + ss0.b_hat_formula_h(h_idx) * mass_retirees_h;
end
report_ss0.PAYG_formula_benefit = total_formula_benefit_ss0;

total_formula_benefit_ssF = 0;
aR_idx_F = cS.aR_new_path(end);
for h_idx = 1:cS.nTypes
    mass_retirees_h = sum(Z_ss_norm_F((aR_idx_F+1):end, h_idx));
    total_formula_benefit_ssF = total_formula_benefit_ssF + ssF.b_hat_formula_h(h_idx) * mass_retirees_h;
end
report_ssF.PAYG_formula_benefit = total_formula_benefit_ssF;

% --- 为报告结构体补充额外的比率 ---
report_ss0.Ig_Y_ratio = (report_ss0.I_g / report_ss0.Y) * 100;
report_ssF.Ig_Y_ratio = (report_ssF.I_g / report_ssF.Y) * 100;
report_ss0.Gc_Y_ratio = (report_ss0.G_c / report_ss0.Y) * 100;
report_ssF.Gc_Y_ratio = (report_ssF.G_c / report_ssF.Y) * 100;

fprintf('   ✅ 核算报告已在内存中生成并补充完毕。\n');


%% --- 3. [核心] 定义比较变量并计算差异 ---
fprintf('\n--- 3. 定义比较变量并计算表格内容 ---\n');

% --- [核心修改] 定义要比较的变量和格式 ---
vars_to_compare = {
    '\multicolumn{4}{l}{\textbf{A. 有效人均宏观总量}}', '', '', '', '';
    '产出 ($y$)', 'Y', '%.4f', 'per_capita', '';
    '私人资本 ($k_p$)', 'K_p', '%.4f', 'per_capita', '';
    '公共资本 ($k_g$)', 'K_g', '%.4f', 'per_capita', '';
    '私人消费 ($c$)', 'C', '%.4f', 'per_capita', '';
    '私人投资 ($i_p$)', 'I_p_acct', '%.4f', 'per_capita', '';
    '\multicolumn{4}{l}{\textbf{B. 核心宏观比率}}', '', '', '', '';
    '私人资本/产出比 ($K_p/Y$)', 'Kp_Y_ratio', '%.3f', 'diff', '';
    '消费/产出比 ($C/Y$)', 'C_Y_ratio', '%.2f', 'ratio_pct', '\%';
    '私人投资/产出比 ($I_p/Y$)', 'Ip_Y_ratio', '%.2f', 'ratio_pct', '\%';
    '公共投资/产出比 ($I_g/Y$)', 'Ig_Y_ratio', '%.2f', 'ratio_pct', '\%';
    '政府消费/产出比 ($G_c/Y$)', 'Gc_Y_ratio', '%.2f', 'ratio_pct', '\%';
    '\multicolumn{4}{l}{\textbf{C. 要素价格}}', '', '', '', '';
    '年化真实利率 ($r$)', 'r_annual', '%.2f', 'diff', '\%';
    '有效工资率 ($w$)', 'w', '%.4f', 'abs_change_pct', '';
    '\multicolumn{4}{l}{\textbf{D. PAYG养老金体系}}', '', '', '', '';
    '总缴费 (占Y比例)', 'PAYG_contrib', '%.2f', 'ratio_y', '\%';
    '规则化总福利 (占Y比例)', 'PAYG_formula_benefit', '%.2f', 'ratio_y', '\%';
    '均衡调整因子 (adj)', 'adj', '%.4f', 'diff_abs', '';
};


% --- 初始化存储结果的 cell 数组 ---
table_data = cell(size(vars_to_compare, 1), 4); % {Label, Val_ss0, Val_ssF, Change}

% --- 循环处理每个变量 ---
for i = 1:size(vars_to_compare, 1)
    label = vars_to_compare{i, 1};
    var_name = vars_to_compare{i, 2};
    
    if isempty(var_name), table_data{i, 1} = label; continue; end
    
    table_data{i, 1} = label;
    
    format_spec = vars_to_compare{i, 3};
    change_type = vars_to_compare{i, 4};
    unit = vars_to_compare{i, 5};
    
    % --- 提取数值 ---
    if strcmp(var_name, 'adj')
        val_ss0 = ss0.adj; val_ssF = ssF.adj;
    else
        if isfield(report_ss0, var_name), val_ss0 = report_ss0.(var_name); else, val_ss0 = NaN; end
        if isfield(report_ssF, var_name), val_ssF = report_ssF.(var_name); else, val_ssF = NaN; end
    end
    
    % --- 根据类型进行预处理 ---
    if strcmp(change_type, 'per_capita')
        val_ss0 = val_ss0 / mass_total_0;
        val_ssF = val_ssF / mass_total_F;
    elseif strcmp(change_type, 'ratio_y')
        if isfield(report_ss0, 'Y') && report_ss0.Y~=0, val_ss0 = (val_ss0 / report_ss0.Y)*100; end
        if isfield(report_ssF, 'Y') && report_ssF.Y~=0, val_ssF = (val_ssF / report_ssF.Y)*100; end
    end
    
    % --- 格式化数值列 ---
    if isnan(val_ss0), table_data{i, 2} = '-'; else, table_data{i, 2} = [sprintf(format_spec, val_ss0), unit]; end
    if isnan(val_ssF), table_data{i, 3} = '-'; else, table_data{i, 3} = [sprintf(format_spec, val_ssF), unit]; end

    % --- 计算并格式化变化列 ---
    change_val_str = '-';
    if ~isnan(val_ss0) && ~isnan(val_ssF)
        switch change_type
            case {'per_capita', 'abs_change_pct'} % 相对变化
                if abs(val_ss0) > 1e-9, change = (val_ssF / val_ss0 - 1) * 100; else, change = NaN; end
                if ~isnan(change), change_val_str = sprintf('%.2f\\%%', change); end
            case {'diff', 'ratio_y', 'ratio_pct'} % 百分点差异
                change = val_ssF - val_ss0;
                change_val_str = sprintf('%.2f pp', change);
            case 'diff_abs' % 绝对值差异
                change = val_ssF - val_ss0;
                change_val_str = sprintf('%+.4f', change);
        end
    end
    table_data{i, 4} = change_val_str;
end
fprintf('   ✅ 表格内容计算完成。\n');

%% --- 4. 生成 LaTeX 表格 ---
fprintf('\n--- 4. 生成 LaTeX 格式的稳态比较表格 ---\n');

% --- 准备输出目录和文件 ---
[output_dir, ~, ~] = fileparts(output_tex_filename);
if ~exist(output_dir, 'dir'), mkdir(output_dir); end
fileID = fopen(output_tex_filename, 'w', 'n', 'UTF-8');
if fileID == -1, error('无法创建或打开LaTex输出文件。'); end

% --- 写入表格框架 ---
fprintf(fileID, '%% =======================================================\n');
fprintf(fileID, '%%  此文件由 ss_comparison.m (v2.2) 自动生成\n');
fprintf(fileID, '%% =======================================================\n');
fprintf(fileID, '\\begin{table}[h!]\n');
fprintf(fileID, '\\centering\n');
fprintf(fileID, '\\caption{转轨路径的初期与终期稳态宏观经济结果比较}\n');
fprintf(fileID, '\\label{tab4.0:ss_comparison}\n');
fprintf(fileID, '\\begin{threeparttable}\n');
fprintf(fileID, '\\begin{tabular}{lccc}\n');
fprintf(fileID, '\\toprule\n');

% [核心修改] 更新表头以包含动态年份
fprintf(fileID, '变量 & 起点 (%d年) & 终期稳态 (%d年) & 变化 \\\\\n', cS.start_year, cS.end_year);

fprintf(fileID, '\\midrule\n');

% --- 循环写入表格内容 ---
for i = 1:size(table_data, 1)
    if isempty(vars_to_compare{i, 2}) % 如果是子标题
        fprintf(fileID, '%s \\\\\n', table_data{i, 1});
    else
        row_str = strjoin(table_data(i,:), ' & ');
        fprintf(fileID, '%s \\\\\n', row_str);
    end
end

% --- 写入表格结尾和注释 ---
fprintf(fileID, '\\bottomrule\n');
fprintf(fileID, '\\end{tabular}\n');
fprintf(fileID, '\\begin{tablenotes}[para,flushleft]\n');
fprintf(fileID, '  \\item 注： “期初稳态”对应模型在转轨路径第一期(t=1)的人口与政策环境下的伪稳态解。“终期稳态”则对应路径最后一期(t=T)的BGP稳态解。\n');
fprintf(fileID, '  除特别注明外，所有流量和存量均为标准化值。财政与养老金流量已表示为占各自稳态产出(Y)的百分比。\n');
fprintf(fileID, '  均衡调整因子(adj)是在保持缴费率固定的前提下，为保证PAYG体系收支平衡所必需的福利调整系数。adj=1.0表示福利可100\\%%兑现。\n');
fprintf(fileID, '  “变化”列中，“\\%%”表示相对变化率，“pp”表示百分点差异。\n');
fprintf(fileID, '\\end{tablenotes}\n');
fprintf(fileID, '\\end{threeparttable}\n');
fprintf(fileID, '\\end{table}\n');

% --- 关闭文件 ---
fclose(fileID);
fprintf('   ✅ 成功生成 LaTeX 表格文件: %s\n', output_tex_filename);
fprintf('\n--- 稳态比较脚本执行完毕 ---\n');

## 图 4.1--人口冲击压力下的宏观经济转轨动态 

## 图 4.2--固定替代率承诺下的社保基金动态与财政压力

In [ ]:
% =========================================================================
% == SCRIPT: plot_combined_analysis.m
% == 版本: [v1.7 - PAYG单位修正与精确定位版]
% ==
% == 目的:
% ==   1. [核心修正] 纠正了图4.2(PAYG流量)的单位错误，通过直接使用
% ==      模型计算出的原始GDP比率进行绘图，移除了不必要的单位转换。
% ==   2. [格式化] 严格按照要求设置两个图窗的'Position'属性，
% ==      以获得精确的尺寸和位置。
% ==   3. 确保所有文本标签均为简体中文。
% ==
% == 前置条件:
% ==   - TRANS/TPI_results_het_nopps_Retire_mild.mat
% =========================================================================

clear; close all;
addpath(pwd);

fprintf('=== 异质性模型综合分析与绘图脚本 (v1.7 - PAYG单位修正版) ===\n\n');

%% --- 1. 用户设定与文件定义 ---
fprintf('--- 1. 用户设定与文件定义 ---\n');

% --- 真实世界单位转换参数 ---
real_gdp_2023 = 126.06; % 2023年中国GDP (万亿元)
annual_inflation_rate = 0.02; % 假设的年化温和通货膨胀率 (2%)

% --- 图4.1 & 4.2 通用设定 ---
year_max = 2100; % 控制图表展示的结束年份

% --- 图4.2 专属设定 ---
exo_rho = 0.40; % 外生目标养老金替代率 (相对于当期有效工资)

% --- 输入文件 ---
file_tpi_results = 'TRANS/TPI_results_het_nopps.mat';

% --- 输出文件 ---
output_dir = 'tex/fig';
output_fig_4_1 = fullfile(output_dir, 'fig4.1_aging_impact_dashboard_het.png');
output_fig_4_2 = fullfile(output_dir, 'fig4.2_exo_pension_fund_het.png');

if ~exist(file_tpi_results, 'file'), error('找不到TPI结果文件: %s', file_tpi_results); end
if ~exist(output_dir, 'dir'), mkdir(output_dir); fprintf('   已创建输出目录: %s\n', output_dir); end

%% --- 2. 数据加载与解包 ---
fprintf('--- 2. 数据加载与解包 ---\n');
fprintf('   正在加载模型转轨结果: %s\n', file_tpi_results);

data_loaded = load(file_tpi_results);
results = data_loaded.results;
cS = data_loaded.cS;
ss0 = data_loaded.ss0;
final_Dist_path_h = data_loaded.final_Dist_path_h;
final_Pol_path_h = data_loaded.final_Pol_path_h;
paramSF = results.ss_data.paramSF;

[~, ~, cS] = population.generate_Z_path(cS, false);
fprintf('   已调用 population.generate_Z_path 更新人口数据。\n');

T = cS.T_sim;
nH = cS.nTypes;
Y_model_base = results.Y_path(1); 

fprintf('   ✅ 数据加载完毕。模型模拟 %d 期, 包含 %d 类家庭。\n', T, nH);
fprintf('   [单位转换基准] 2023年GDP=%.2f万亿元, 年通胀率=%.1f%%\n', real_gdp_2023, annual_inflation_rate*100);

%% --- 3. [图4.1] 老龄化冲击下的宏观经济动态仪表盘 ---
fprintf('\n--- 3. [生成图4.1] 老龄化宏观影响仪表盘 ---\n');

% --- 3.1 准备绘图数据 ---
fprintf('   正在为图4.1准备数据 (含真实单位转换)...\n');
paths_4_1 = struct();

time_axis_full = (cS.start_year : cS.time_Step : (cS.start_year + (T-1)*cS.time_Step));
time_mask = time_axis_full <= year_max;
time_axis = time_axis_full(time_mask);
total_pop_path = sum(cS.Z_path_raw, 1) * 1000;
pop_path_masked = total_pop_path(time_mask);

K_real_agg = convertToRealWorldUnits(results.K_p_path(time_mask), time_axis, cS, Y_model_base, real_gdp_2023, annual_inflation_rate, 'trillion');
Y_real_agg = convertToRealWorldUnits(results.Y_path(time_mask), time_axis, cS, Y_model_base, real_gdp_2023, annual_inflation_rate, 'trillion');
paths_4_1.K_pc_real = (K_real_agg ./ pop_path_masked) * 1e8;
paths_4_1.Y_pc_real = (Y_real_agg ./ pop_path_masked) * 1e8;

aggr_supply = aggregates.get_path_aggregates(final_Dist_path_h, final_Pol_path_h, cS, paramSF);
L_path_hat = aggr_supply.L_path;
total_labor_income_level = results.w_path .* L_path_hat;
per_capita_labor_income_level = total_labor_income_level ./ total_pop_path;
paths_4_1.w_pc_real = convertToRealWorldUnits(per_capita_labor_income_level(time_mask), time_axis, cS, Y_model_base, real_gdp_2023, annual_inflation_rate, 'ten_thousand');

paths_4_1.r_path = results.r_path(time_mask);
mass_retirees_path = zeros(1, T);
for t = 1:T
    aR_t = cS.aR_new_path(t);
    mass_retirees_path(t) = sum(cS.Z_path_raw((aR_t+1):end, t));
end
paths_4_1.retiree_share = mass_retirees_path(time_mask) ./ (total_pop_path(time_mask)/1000);
fprintf('   ✅ 数据准备完毕。\n');

% --- 3.2 绘图 ---
fprintf('   正在生成图表 (1x3布局)...\n');
fig1 = figure('Name', '老龄化冲击下的宏观经济动态(真实单位)', 'Position', [100 679 787 241]);
tcl1 = tiledlayout(1, 3, 'Padding', 'compact', 'TileSpacing', 'normal');

style_K_pc = {'k-', 'LineWidth', 2.5}; 
style_Y_pc = {'--', 'Color', [0.4 0.4 0.4], 'LineWidth', 2.5}; 
style_w_pc = {'k-', 'LineWidth', 2.5};
style_r    = {'k-', 'LineWidth', 2.5};
style_bg_line  = {'-.', 'Color', [0.8 0.8 0.8], 'LineWidth', 2};
color_left_axis = 'black';
color_right_axis = [0.5 0.5 0.5];
font_size_title = 14;
font_size_axis  = 11;
yy_lim_bar = [15 80];
tick_start_year = ceil(time_axis(1)/50)*50;
xticks_vec = tick_start_year :30 : time_axis(end);
xticks_vec = unique([time_axis(1), xticks_vec]);

ax1 = nexttile;
hold(ax1, 'on'); set(ax1, 'FontName', 'Times New Roman', 'FontSize', font_size_axis);
yyaxis(ax1, 'right');
p_bg_line1 = plot(ax1, time_axis, paths_4_1.retiree_share * 100, style_bg_line{:}, 'DisplayName', ['退休人口占比' newline '(右轴, %)']);
set(ax1, 'YColor', color_right_axis, 'YLim', yy_lim_bar);
yyaxis(ax1, 'left');
p_K_pc = plot(ax1, time_axis, paths_4_1.K_pc_real, style_K_pc{:}, 'DisplayName', '人均私人资本');
p_Y_pc = plot(ax1, time_axis, paths_4_1.Y_pc_real, style_Y_pc{:}, 'DisplayName', '人均产出');
set(ax1, 'YColor', color_left_axis); ylabel(ax1, '万元', 'FontName', 'SimSun'); ylim(ax1, 'padded');
title(ax1, '(a) 人均资本与产出', 'FontSize', font_size_title, 'FontName', 'SimSun');
legend(ax1, [p_K_pc, p_Y_pc, p_bg_line1], 'Location', 'northwest', 'FontSize', 10, 'Box', 'off', 'FontName', 'SimSun');
grid(ax1, 'on'); xlim(ax1, [time_axis(1), time_axis(end)]); 
set(ax1, 'XTick', 2030:10:2100); % 根据图像调整刻度
xtickangle(ax1, 40);
hold(ax1, 'off');

ax2 = nexttile;
hold(ax2, 'on'); set(ax2, 'FontName', 'Times New Roman', 'FontSize', font_size_axis);
yyaxis(ax2, 'right');
p_bg_line2 = plot(ax2, time_axis, paths_4_1.retiree_share * 100, style_bg_line{:});
set(ax2, 'YColor', color_right_axis, 'YLim', yy_lim_bar);
yyaxis(ax2, 'left');
plot(ax2, time_axis, paths_4_1.w_pc_real, style_w_pc{:});
set(ax2, 'YColor', color_left_axis); ylabel(ax2, '万元', 'FontName', 'SimSun'); ylim(ax2, 'padded');
title(ax2, '(b) 人均劳动收入', 'FontSize', font_size_title, 'FontName', 'SimSun');
legend(ax2, p_bg_line2, ['退休人口占比' newline '(右轴, %)'], 'Location', 'southeast', 'FontSize', 10, 'Box', 'off', 'FontName', 'SimSun');
grid(ax2, 'on'); xlim(ax2, [time_axis(1), time_axis(end)]); 
set(ax2, 'XTick', 2030:10:2100); % 根据图像调整刻度
xtickangle(ax2, 40);
hold(ax2, 'off');

ax3 = nexttile;
hold(ax3, 'on'); set(ax3, 'FontName', 'Times New Roman', 'FontSize', font_size_axis);
yyaxis(ax3, 'right');
p_bg_line3 = plot(ax3, time_axis, paths_4_1.retiree_share * 100, style_bg_line{:});
set(ax3, 'YColor', color_right_axis, 'YLim', yy_lim_bar);
yyaxis(ax3, 'left');
r_annual = (1 + paths_4_1.r_path).^(1/cS.time_Step) - 1;
plot(ax3, time_axis, r_annual * 100, style_r{:});
set(ax3, 'YColor', color_left_axis); ylim(ax3, 'padded');
title(ax3, '(c) 年化真实利率 (%)', 'FontSize', font_size_title, 'FontName', 'SimSun');
legend(ax3, p_bg_line3, ['退休人口占比' newline '(右轴, %)'], 'Location', 'northeast', 'FontSize', 10, 'Box', 'off', 'FontName', 'SimSun');
grid(ax3, 'on'); xlim(ax3, [time_axis(1), time_axis(end)]); 
set(ax3, 'XTick', 2030:10:2100); % 根据图像调整刻度
xtickangle(ax3, 40);
hold(ax3, 'off');

fprintf('   正在保存图像至: %s\n', output_fig_4_1);
try
    exportgraphics(fig1, output_fig_4_1, 'Resolution', 600);
    fprintf('   ✅ 图4.1保存成功。\n');
catch ME
    fprintf('   ❌ 图像保存失败！错误信息: %s\n', ME.message);
end

%% --- 4. [图4.2] PAYG养老金收支流量分析 ---
fprintf('\n--- 4. [生成图4.2] PAYG收支流量分析 ---\n');

% --- 4.1 会计核算 (不进行真实世界单位转换) ---
fprintf('   正在进行PAYG收支流量的会计核算 (原始模型单位)...\n');
iter_results = results.iter_results;
adj_path = iter_results.adj_path_iter;
w_hat_path_iter = iter_results.w_hat_path_iter;
w_hat_path_for_b = [ss0.w_hat, w_hat_path_iter(1:T-1)];
b_hat_formula_path_h = zeros(nH, T);
for t = 1:T
    b_hat_formula_path_h(:, t) = SS.calculate_formula_benefits(w_hat_path_for_b(t), cS);
end
b_hat_path_h_actual = adj_path .* b_hat_formula_path_h;

Y_path_level = results.Y_path;

mass_retirees_path_h = zeros(nH, T);
for t = 1:T
    aR_t = cS.aR_new_path(t);
    mass_retirees_total_t = sum(cS.Z_path_raw((aR_t+1):end, t));
    for h = 1:nH
        mass_retirees_path_h(h, t) = mass_retirees_total_t * cS.type_weights(h);
    end
end

b_hat_path_exo = exo_rho * w_hat_path_iter;

Total_PAYG_Contrib_level = sum(b_hat_path_h_actual .* cS.A_path .* mass_retirees_path_h, 1);
Total_Promised_Benefit_level = sum(repmat(b_hat_path_exo, nH, 1) .* cS.A_path .* mass_retirees_path_h, 1);

% [核心修正] 直接计算原始比率，不进行任何单位转换
contrib_gdp_ratio = Total_PAYG_Contrib_level ./ Y_path_level;
benefit_gdp_ratio = Total_Promised_Benefit_level ./ Y_path_level;
fprintf('   ✅ 会计核算完成。\n');

% --- 4.2 准备绘图数据 ---
fprintf('   正在为图4.2准备数据...\n');
paths_4_2 = struct();
paths_4_2.contrib_gdp = contrib_gdp_ratio(time_mask);
paths_4_2.benefit_gdp = benefit_gdp_ratio(time_mask);
fprintf('   ✅ 数据准备完毕。\n');

% --- 4.3 绘图 ---
fprintf('   正在生成图表...\n');
fig2 = figure('Name', 'PAYG收支流量分析', 'Position', [461 407 550 243]);

ax = gca;
hold(ax, 'on');
style_contrib = {'k-', 'LineWidth', 2};
style_benefit = {'--', 'Color', [0.4 0.4 0.4], 'LineWidth', 2};
color_deficit_area = [0.8 0.8 0.8];
set(ax, 'FontName', 'Times New Roman', 'FontSize', 11);

deficit_gdp = paths_4_2.benefit_gdp - paths_4_2.contrib_gdp;
p_area = area(ax, time_axis, deficit_gdp * 100, ...
    'FaceColor', color_deficit_area, 'FaceAlpha', 0.6, ...
    'EdgeColor', 'none', 'DisplayName', 'PAYG缺口');
p_contrib = plot(ax, time_axis, paths_4_2.contrib_gdp * 100, style_contrib{:}, 'DisplayName', 'PAYG缴费收入');
p_benefit = plot(ax, time_axis, paths_4_2.benefit_gdp * 100, style_benefit{:}, 'DisplayName', '承诺的福利支出');

ylim(ax, [0, max(paths_4_2.benefit_gdp)*100*1.1]);
ylabel(ax, '占GDP的百分比 (%)', 'FontName', 'SimSun');
xlabel(ax, '年份', 'FontName', 'SimSun');

title_obj = title(ax, 'PAYG收支流量占GDP比重', 'FontSize', 14, 'FontName', 'SimSun');
legend_obj = legend(ax, [p_contrib, p_benefit, p_area], 'Location', 'northwest', 'FontSize', 10, 'Box', 'off', 'FontName', 'SimSun');

grid(ax, 'on'); box(ax, 'on');
xlim(ax, [time_axis(1), time_axis(end)]);
set(ax, 'XTick', 2030:10:2100); % 根据图像调整刻度
xtickangle(ax, 40);
hold(ax, 'off');

fprintf('   ✅ 图表生成完毕。\n');

% --- 4.4 保存图像 ---
fprintf('   正在保存图像至: %s\n', output_fig_4_2);
try
    exportgraphics(fig2, output_fig_4_2, 'Resolution', 600);
    fprintf('   ✅ 图4.2保存成功。\n');
catch ME
    fprintf('   ❌ 图像保存失败！错误信息: %s\n', ME.message);
end

fprintf('\n--- 脚本执行完毕 ---\n');

%% --- [本地辅助函数] ---
function real_world_path = convertToRealWorldUnits(model_path, time_axis, cS, Y_model_base, real_gdp_base, inflation_rate, target_unit)
    scale_factor = real_gdp_base / Y_model_base;
    real_path = model_path * scale_factor;
    years_since_start = time_axis - cS.start_year;
    price_level_path = (1 + inflation_rate) .^ years_since_start;
    nominal_path = real_path .* price_level_path;
    
    switch lower(target_unit)
        case 'trillion' 
            real_world_path = nominal_path;
        case 'ten_thousand' 
            real_world_path = nominal_path * 1e8;
        otherwise
            error('未知的目标单位: %s。请使用 "trillion" 或 "ten_thousand"。', target_unit);
    end
end

## 表 4.1--各类型家庭相对于高收入城镇职工的福利差距

In [ ]:
% =========================================================================
% == SCRIPT: create_welfare_table.m
% == 版本: [v1.0 - 跨类型福利差距表格生成器]
% ==
% == 目的:
% ==   1. 加载单一PAYG路径下的异质性家庭模型结果。
% ==   2. 比较中低收入家庭及居民家庭相对于高收入城镇职工家庭的福利水平。
% ==   3. 按工作年龄和退休年龄两个年龄段，计算关键年份的平均福利差距(CV)。
% ==   4. 生成LaTeX表格 (Tab4.1)，以论证PAYG福利削减对不同群体的非对称影响。
% ==
% == 前置条件:
% ==   - 已成功运行 main_run_trans.m 并生成以下文件:
% ==     - TRANS/TPI_results_het_nopps.mat
% =========================================================================

clear; close all; clc;
addpath(pwd);
fprintf('=== 跨类型福利差距表格生成脚本 (v1.0) ===\n\n');

%% --- 1. 用户设定与数据加载 ---
fprintf('--- 1. 用户设定与文件定义 ---\n');

% --- 输入文件 ---
input_file = 'TRANS/TPI_results_het_nopps_raw.mat';

% --- 输出路径 ---
output_dir = 'tex/tab/';
if ~exist(output_dir, 'dir'), mkdir(output_dir); end
output_filename = fullfile(output_dir, 'tab4.1_welfare_gap_across_types.tex');

% --- [核心] 表格分析维度 ---
assessment_years = [2025, 2040, 2055, 2070, 2085, 2100];

% --- 加载数据 ---
fprintf('   正在加载文件: %s ...\n', input_file);
if ~exist(input_file, 'file'), error('找不到文件: %s', input_file); end
data = load(input_file);
cS = data.cS;
ss0 = data.ss0;
paramSF = data.results.ss_data.paramSF;
valF_h = data.results.ss_data.valF_h;
polF_h = data.results.ss_data.polF_h;
iter_results = data.results.iter_results;
dist_path_h = data.final_Dist_path_h;
fprintf('   ✅ 数据加载完成。\n');


%% --- 2. [关键步骤] 重构价值函数路径 ---
fprintf('\n--- 2. 正在重构价值函数路径 (可能需要一些时间) ---\n');

Val_path_h = cell(cS.nTypes, 1);
for h = 1:cS.nTypes
    pathS_h = get_pathS_for_backward(iter_results, h, cS, ss0);
    cS_h = get_cs_for_type(cS, h);
    [~, Val_path_h{h}] = household.backward_hh(pathS_h, cS_h, paramSF, valF_h{h}, polF_h{h});
end
fprintf('   ✅ 所有家庭类型的价值函数路径重构完成。\n');


%% --- 3. 计算跨类型福利差距(CV) ---
fprintf('\n--- 3. 正在计算跨类型福利差距(CV) ---\n');

% --- [核心] 定义新的年龄组 ---
age1 = cS.age1_orig;
step = cS.time_Step;
age_groups = {
    'working', '工作年龄人口 (20-60岁)', floor((20-age1)/step)+1 : floor((60-age1)/step)+1;
    'retired', '退休人口 (61岁及以上)', floor((61-age1)/step)+1 : cS.aD_new
};
age_groups(:,3) = cellfun(@(x) x(x>=1 & x<=cS.aD_new), age_groups(:,3), 'UniformOutput', false);

T = cS.T_sim;
sigma = cS.sigma;
num_comparison_types = cS.nTypes - 1;

% --- 初始化存储结果的容器 ---
cv_results = struct();
for i = 1:size(age_groups, 1)
    group_identifier = age_groups{i, 1};
    cv_results.(group_identifier) = zeros(num_comparison_types, T);
end

% --- 核心计算循环 ---
for t = 1:T
    for i_group = 1:size(age_groups, 1)
        group_identifier = age_groups{i_group, 1};
        age_indices = age_groups{i_group, 3};
        
        avg_ev_by_type = zeros(cS.nTypes, 1);
        for h = 1:cS.nTypes
            total_ev = 0; total_pop = 0;
            for a_idx = age_indices
                dist_slice = dist_path_h{h}(:, :, :, a_idx, t);
                val_slice  = Val_path_h{h}(:, :, :, a_idx, t);
                total_ev = total_ev + sum(val_slice .* dist_slice, 'all');
                total_pop = total_pop + sum(dist_slice, 'all');
            end
            if total_pop > 1e-12, avg_ev_by_type(h) = total_ev / total_pop; end
        end
        
        ev_benchmark = avg_ev_by_type(1);
        for h_comp = 1:num_comparison_types
            h = h_comp + 1;
            ev_comp = avg_ev_by_type(h);
            cv = NaN;
            if ev_benchmark < -1e-9 && ev_comp < -1e-9
                cv = (ev_comp / ev_benchmark)^(1 / (1 - sigma)) - 1;
            end
            cv_results.(group_identifier)(h_comp, t) = cv;
        end
    end
end
fprintf('   ✅ 福利差距计算完成。\n');

%% --- 4. [核心] 生成 LaTeX 表格 ---
fprintf('\n--- 4. 生成 LaTeX 格式的福利差距表格 ---\n');

% --- 准备输出目录和文件 ---
fileID = fopen(output_filename, 'w', 'n', 'UTF-8');
if fileID == -1, error('无法创建或打开LaTex输出文件。'); end

% --- 写入表格框架 ---
fprintf(fileID, '%% =======================================================\n');
fprintf(fileID, '%%  此文件由 create_welfare_table.m (v1.0) 自动生成\n');
fprintf(fileID, '%% =======================================================\n');
fprintf(fileID, '\\begin{table}[h!]\n');
fprintf(fileID, '\\centering\n');
fprintf(fileID, '\\caption{各类型家庭相对于高收入城镇职工的福利差距 (\\%%)}\n');
fprintf(fileID, '\\label{tab4.1:welfare_gap}\n');
fprintf(fileID, '\\begin{threeparttable}\n');

% 动态生成 tabular 列定义
col_def = ['l', repmat('c', 1, length(assessment_years))];
fprintf(fileID, '\\begin{tabular}{%s}\n', col_def);
fprintf(fileID, '\\toprule\n');

% 动态生成表头
header_str = '家庭类型与年龄组';
for year = assessment_years
    header_str = [header_str, sprintf(' & %d年', year)];
end
fprintf(fileID, '%s \\\\\n', header_str);
fprintf(fileID, '\\midrule\n');

% --- 循环写入表格内容 ---
time_axis_full = cS.start_year : cS.time_Step : (cS.start_year + (T-1)*cS.time_Step);
type_labels = {'中低收入城镇职工', '高收入居民', '中低收入居民'};

for i_group = 1:size(age_groups, 1)
    group_identifier = age_groups{i_group, 1};
    group_label_cn = age_groups{i_group, 2};
    
    % 打印年龄组子标题
    fprintf(fileID, '\\multicolumn{%d}{l}{\\textbf{%s}} \\\\\n', 1 + length(assessment_years), group_label_cn);
    
    for h_comp = 1:num_comparison_types
        % 打印家庭类型标签
        fprintf(fileID, '    \\hspace{1em}%s', type_labels{h_comp});
        
        % 循环写入该行每一年的CV值
        for year = assessment_years
            t_idx = round((year - cS.start_year) / cS.time_Step) + 1;
            
            cv_str = '-';
            if t_idx > 0 && t_idx <= T
                cv_value = cv_results.(group_identifier)(h_comp, t_idx);
                if ~isnan(cv_value)
                    cv_str = sprintf('%.2f\\%%', cv_value * 100);
                end
            end
            fprintf(fileID, ' & %s', cv_str);
        end
        fprintf(fileID, ' \\\\\n');
    end
    if i_group < size(age_groups, 1), fprintf(fileID, '\\addlinespace\n'); end
end

% --- 写入表格结尾和注释 ---
fprintf(fileID, '\\bottomrule\n');
fprintf(fileID, '\\end{tabular}\n');
fprintf(fileID, '\\begin{tablenotes}[para,flushleft]\n');
fprintf(fileID, '  \\item 注： 表中数值为补偿变化(CV)，衡量各群体相对于“高收入城镇职工”群体的福利差距。例如，-5.00\\%%表示该群体愿意放弃其生命周期消费的5.0\\%%，以交换进入高收入城镇职工的福利水平。负值越大，福利差距越大。\n');
fprintf(fileID, '\\end{tablenotes}\n');
fprintf(fileID, '\\end{threeparttable}\n');
fprintf(fileID, '\\end{table}\n');

% --- 关闭文件 ---
fclose(fileID);
fprintf('   ✅ 成功生成 LaTeX 表格文件: %s\n', output_filename);
fprintf('\n--- 脚本执行完毕 ---\n');


%% --- [本地辅助函数] ---

function pathS_h = get_pathS_for_backward(iter_results, h, cS, ss0)
    % 为指定的家庭类型 h，打包后向迭代所需的所有时间路径
    pathS_h = struct();
    pathS_h.r_path = iter_results.r_path_iter;
    w_hat_path_converged = iter_results.w_hat_path_iter;
    pathS_h.w_hat_path = w_hat_path_converged;
    pathS_h.tr_per_hh_hat_path = iter_results.TR_hat_pc_path_iter;
    
    w_hat_path_for_b = [ss0.w_hat, w_hat_path_converged(1:end-1)];
    b_hat_formula_path_h_all = zeros(cS.nTypes, cS.T_sim);
    for t = 1:cS.T_sim
        b_hat_formula_path_h_all(:, t) = SS.calculate_formula_benefits(w_hat_path_for_b(t), cS);
    end
    adj_path_converged = iter_results.adj_path_iter;
    pathS_h.b_hat_path = adj_path_converged .* b_hat_formula_path_h_all(h, :);
    
    A_path_ext = [cS.A_path, cS.A_path(end) * (1 + ((1+cS.g_A_ss)^cS.time_Step-1))];
    g_A_path = A_path_ext(2:end)./A_path_ext(1:end-1)-1;
    pathS_h.g_A_path = g_A_path(1:cS.T_sim);
    pathS_h.theta_path = cS.theta_path_h(h, :);
end

function cS_h = get_cs_for_type(cS, h)
    % 为指定的家庭类型 h，准备一个专属的 cS 参数包
    cS_h = cS;
    cS_h.ageEffV_new = cS.ageEffV_new_h(:, h);
end

# 第五节：改革路径

## 延迟退休

### 图 4.5--延迟退休改革路径、抚养比与现收现付缺口

### 图 4.6--典型年龄组在不同延迟退休变革方案下的福利影响比较

In [ ]:
% =========================================================================
% == SCRIPT: analyze_retirement_reforms.m
% ==
% == 版本: [v2.0 - 分析视角与可视化重构]
% ==
% == 目的:
% ==   1. [分析视角] 将福利分析的基准情景从'moderate'改为'mild'。
% ==   2. [年龄组调整] 福利分析聚焦于45-59岁的三个中年及临近退休年龄组。
% ==   3. [可视化重构] 图1改为直接展示退休年龄路径和PAYG收支缺口。
% ==   4. [风格统一] 所有图表统一为灰度风格。
% =========================================================================

clear; close all; clc;
addpath(pwd);
fprintf('=== 延迟退休改革多情景对比分析脚本 (v2.0 - 视角重构版) ===\n\n');

%% --- 1. 用户设定与文件定义 ---
fprintf('--- 1. 用户设定与文件定义 ---\n');

% --- 分析设定 ---
scenarios = {'mild','moderate', 'aggressive'};
% [核心修改] 将福利分析的基准改为'mild'
base_scenario_for_welfare = 'mild'; 
initial_fund_to_gdp_ratio = 0.0239;
year_max = 2100;

% --- 文件路径模板 (仅需TPI结果文件) ---
file_tpi_pattern = 'TRANS/TPI_results_het_nopps_Retire_%s.mat';

% --- 输出路径 ---
output_dir = 'tex/fig/';
if ~exist(output_dir, 'dir'), mkdir(output_dir); end
output_fig_policy_deficit = fullfile(output_dir, 'fig4.5_retire_reform_policy_deficit.png');
output_fig_welfare = fullfile(output_dir, 'fig4.6_retire_reform_welfare_impact.png');

fprintf('   基准方案 (福利): %s\n', base_scenario_for_welfare);
fprintf('   初始基金/GDP比例: %.2f%%\n', initial_fund_to_gdp_ratio * 100);

%% --- 2. [宏观分析] 加载数据并计算核心宏观路径 ---
fprintf('\n--- 2. [宏观分析] 循环加载各方案，计算核心宏观路径 ---\n');

results_all = struct(); % 用于存储所有方案的核心计算结果

for i = 1:length(scenarios)
    scenario_name = scenarios{i};
    fprintf('\n   正在处理方案: [%s] ...\n', upper(scenario_name));

    % --- a. 加载数据 ---
    file_tpi = sprintf(file_tpi_pattern, scenario_name);
    if ~exist(file_tpi, 'file'), error('找不到文件: %s', file_tpi); end

    data_tpi = load(file_tpi);
    cS = data_tpi.cS;
    
    % 从加载的数据中获取路径和参数
    Y_path_level = data_tpi.results.Y_path;
    w_path_level = data_tpi.results.w_path;
    dist_path_h = data_tpi.final_Dist_path_h;
    ss_data = data_tpi.results.ss_data;
    paramSF = ss_data.paramSF;

    % [核心修改] 存储绘图和后续分析所需的数据
    results_all.(scenario_name).data_tpi = data_tpi;
    results_all.(scenario_name).Y_path_level = Y_path_level;
    results_all.(scenario_name).aR_new_path = cS.aR_new_path; % 存储退休年龄索引路径

    % --- b. 重构收支路径以计算缺口 ---
    w_hat_path = w_path_level ./ cS.A_path; % 需要 hat 量

    [Total_Contributions_Level, Total_Formula_Benefits_Level] = ...
        recalculate_payg_flows_from_tpi_logic(w_hat_path, dist_path_h, cS, paramSF, ss_data.ss0);

    % === 计算PAYG年度缺口占GDP的比重 ===
    Deficit_Level = Total_Formula_Benefits_Level - Total_Contributions_Level;
    Deficit_to_GDP_path = Deficit_Level ./ Y_path_level;
    results_all.(scenario_name).Deficit_to_GDP_path = Deficit_to_GDP_path;
    
    fprintf('   ✅ 核心宏观路径处理完成。\n');
end

%% --- 3. [微观分析] 计算比较福利 (CV) ---
fprintf('\n--- 3. [微观分析] 计算各方案相对基准的福利变化 ---\n');

% --- a. 定义劳动负效用参数 ---
add_labor_disutility = false; 
labor_disutil_param_psi = 1.5; 

if add_labor_disutility
    fprintf('   *** 将在福利评估中加入劳动负效用 (ψ = %.2f) ***\n', labor_disutil_param_psi);
else
    fprintf('   *** 将在福利评估中【不】加入劳动负效用 ***\n');
end


% --- b. 重构基准方案 ('mild') 的价值和策略路径 ---
fprintf('   正在重构基准方案 [%s] 的路径...\n', upper(base_scenario_for_welfare));
base_results = results_all.(base_scenario_for_welfare);
cS_base = base_results.data_tpi.cS;
iter_results_base = base_results.data_tpi.results.iter_results;
ss_data_base = base_results.data_tpi.results.ss_data;
paramSF_base = ss_data_base.paramSF;
valF_base_h = ss_data_base.valF_h;
polF_base_h = ss_data_base.polF_h;

Val_path_base_h = cell(cS_base.nTypes, 1);
Pol_path_base_h = cell(cS_base.nTypes, 1);
for h = 1:cS_base.nTypes
    pathS_base_h = get_pathS_for_backward(base_results.data_tpi.results, iter_results_base, h, cS_base, ss_data_base.ss0);
    cS_h_base = get_cs_for_type(cS_base, h);
    [Pol_path_base_h{h}, Val_path_base_h{h}] = household.backward_hh(pathS_base_h, cS_h_base, paramSF_base, valF_base_h{h}, polF_base_h{h});
end

if add_labor_disutility
    Val_path_base_for_CV_h = cell(cS_base.nTypes, 1);
    for h = 1:cS_base.nTypes
        cS_h_base = get_cs_for_type(cS_base, h);
        Val_path_base_for_CV_h{h} = recalculate_vpath_with_labor_disutility(...
            Val_path_base_h{h}, Pol_path_base_h{h}, cS_h_base, paramSF_base, labor_disutil_param_psi);
    end
else
    Val_path_base_for_CV_h = Val_path_base_h;
end
fprintf('   ✅ 基准方案路径重构完成。\n');


% --- c. 循环计算各改革方案相对基准的CV ---
reform_scenarios_for_welfare = setdiff(scenarios, base_scenario_for_welfare);
cv_results = struct();

for i = 1:length(reform_scenarios_for_welfare)
    reform_name = reform_scenarios_for_welfare{i};
    fprintf('   正在计算方案 [%s] 相对于 [%s] 的福利...\n', upper(reform_name), upper(base_scenario_for_welfare));
    
    reform_results = results_all.(reform_name);
    cS_reform = reform_results.data_tpi.cS;
    iter_results_reform = reform_results.data_tpi.results.iter_results;
    ss_data_reform = reform_results.data_tpi.results.ss_data;
    paramSF_reform = ss_data_reform.paramSF;
    valF_reform_h = ss_data_reform.valF_h;
    polF_reform_h = ss_data_reform.polF_h;

    Val_path_reform_h = cell(cS_reform.nTypes, 1);
    Pol_path_reform_h = cell(cS_reform.nTypes, 1);
    for h = 1:cS_reform.nTypes
        pathS_reform_h = get_pathS_for_backward(reform_results.data_tpi.results, iter_results_reform, h, cS_reform, ss_data_reform.ss0);
        cS_h_reform = get_cs_for_type(cS_reform, h);
        [Pol_path_reform_h{h}, Val_path_reform_h{h}] = household.backward_hh(pathS_reform_h, cS_h_reform, paramSF_reform, valF_reform_h{h}, polF_reform_h{h});
    end

    if add_labor_disutility
        Val_path_reform_for_CV_h = cell(cS_reform.nTypes, 1);
        for h = 1:cS_reform.nTypes
            cS_h_reform = get_cs_for_type(cS_reform, h);
            Val_path_reform_for_CV_h{h} = recalculate_vpath_with_labor_disutility(...
                Val_path_reform_h{h}, Pol_path_reform_h{h}, cS_h_reform, paramSF_reform, labor_disutil_param_psi);
        end
    else
        Val_path_reform_for_CV_h = Val_path_reform_h;
    end
    
    % [核心修改] 调整福利分析的年龄组
    age_groups_for_figure = {
        '中年I (45-49岁)', round((45 - cS_base.age1_orig)/cS_base.time_Step) + 1;
        '中年II (50-54岁)', round((50 - cS_base.age1_orig)/cS_base.time_Step) + 1;
        '临近退休 (55-59岁)', round((55 - cS_base.age1_orig)/cS_base.time_Step) + 1
    };
    num_age_groups_fig = size(age_groups_for_figure, 1);
    
    T_max_comp = min(cS_base.T_sim, cS_reform.T_sim);
    
    cv_paths_all_types = zeros(num_age_groups_fig, T_max_comp);

    for i_age_grp = 1:num_age_groups_fig
        a_idx = age_groups_for_figure{i_age_grp, 2};
        if a_idx > cS_base.aD_new, continue; end

        for t = 1:T_max_comp
            total_mass_t = 0;
            weighted_EV_base_t = 0;
            weighted_EV_reform_t = 0;
            
            for h = 1:cS_base.nTypes
                dist_base_slice = base_results.data_tpi.final_Dist_path_h{h}(:, :, :, a_idx, t);
                mass_slice = sum(dist_base_slice, 'all');

                if mass_slice > 1e-12
                   total_mass_t = total_mass_t + mass_slice;
                   val_base_slice = Val_path_base_for_CV_h{h}(:, :, :, a_idx, t);
                   val_reform_slice = Val_path_reform_for_CV_h{h}(:, :, :, a_idx, t);
                   weighted_EV_base_t = weighted_EV_base_t + sum(val_base_slice .* dist_base_slice, 'all');
                   weighted_EV_reform_t = weighted_EV_reform_t + sum(val_reform_slice .* dist_base_slice, 'all');
                end
            end
            
            if total_mass_t < 1e-9
                cv_paths_all_types(i_age_grp, t) = 0;
                continue;
            end

            EV_base_avg = weighted_EV_base_t / total_mass_t;
            EV_reform_avg = weighted_EV_reform_t / total_mass_t;
            
            if EV_base_avg > -1e-9 || EV_reform_avg > -1e-9
                cv_paths_all_types(i_age_grp, t) = NaN;
                continue;
            end

            cv = (EV_reform_avg / EV_base_avg)^(1 / (1 - cS_base.sigma)) - 1;
            
            if ~isreal(cv)
                cv_paths_all_types(i_age_grp, t) = NaN;
            else
                cv_paths_all_types(i_age_grp, t) = cv;
            end
        end
    end
    cv_results.(reform_name) = cv_paths_all_types;
    fprintf('   ✅ 福利计算完成。\n');
end
fprintf('--- 福利分析全部完成 ---\n');


%% --- 4. 绘图 ---
fprintf('\n--- 4. 生成结果图表 ---\n');

% --- 设定灰度颜色和线型 ---
color_mild = [0.65 0.65 0.65]; 
color_moderate = [0.35 0.35 0.35]; 
color_aggressive = [0.0 0.0 0.0];
colors = containers.Map(scenarios, {color_mild, color_moderate, color_aggressive});
line_styles = {'--', '-', ':'}; 
line_styles_map = containers.Map(scenarios, line_styles);

% --- 图1: 政策路径、抚养比与PAYG收支缺口 ---
fig1 = figure('Name', '政策路径、抚养比与PAYG缺口', 'Position', [100 100 950 280]); 
tcl1 = tiledlayout(1, 3, 'Padding', 'compact', 'TileSpacing', 'compact'); 

cS_plot = results_all.(scenarios{1}).data_tpi.cS;
time_axis_model = (cS_plot.start_year : cS_plot.time_Step : (cS_plot.start_year + (cS_plot.T_sim-1)*cS_plot.time_Step));
time_axis_annual = (cS_plot.start_year : 1 : min(year_max, time_axis_model(end)));
% -- Panel (a): 平均法定退休年龄路径 (精确复现年度路径) --
ax1a = nexttile;
hold(ax1a, 'on');

legend_policy = struct();
legend_policy.mild = '平缓(我国现行)';
legend_policy.moderate = '温和';
legend_policy.aggressive = '激进';

annual_years_vector = cS_plot.start_year : min(year_max, cS_plot.end_year);
for i = 1:length(scenarios)
    s = scenarios{i};
    % --- 根据方案名称设定改革参数 (与utils.retire_age_path保持一致) ---
    if strcmpi(s, 'mild')
        reform_start_year = 2025; initial_retire_age_male = 60; initial_retire_age_female = 55;
        target_retire_age = 67; annual_increment_years = 0.25;
    elseif strcmpi(s, 'moderate')
        reform_start_year = 2025; initial_retire_age_male = 60; initial_retire_age_female = 55;
        target_retire_age = 67; annual_increment_years = 1/3;
    elseif strcmpi(s, 'aggressive')
        reform_start_year = 2025; initial_retire_age_male = 60; initial_retire_age_female = 55;
        target_retire_age = 67; annual_increment_years = 0.5;
    end
    
    % [核心修改] 创建新的图例标签
    months_per_year = annual_increment_years * 12;
    legend_label = sprintf('%s(%.1d个月/年)', getfield(legend_policy,s), months_per_year);

    % --- 生成男女各自的年度路径 ---
    retire_age_male_annual_path = ones(size(annual_years_vector)) * initial_retire_age_male;
    retire_age_female_annual_path = ones(size(annual_years_vector)) * initial_retire_age_female;
    for idx = 2:length(annual_years_vector)
        year = annual_years_vector(idx);
        if year >= reform_start_year
            new_age_male = retire_age_male_annual_path(idx-1) + annual_increment_years;
            retire_age_male_annual_path(idx) = min(target_retire_age, new_age_male);
            new_age_female = retire_age_female_annual_path(idx-1) + annual_increment_years;
            retire_age_female_annual_path(idx) = min(target_retire_age, new_age_female);
        end
    end
    % --- 计算加权平均并绘图 ---
    avg_retire_age_annual_path = 0.5 * retire_age_male_annual_path + 0.5 * retire_age_female_annual_path;
    % [核心修改] 使用新的图例标签
    plot(ax1a, annual_years_vector, avg_retire_age_annual_path, 'LineWidth', 2.5, 'Color', colors(s), 'LineStyle', line_styles_map(s), 'DisplayName', legend_label);
end
hold(ax1a, 'off');
grid(ax1a, 'on'); box(ax1a, 'on');
xlim(ax1a, [annual_years_vector(1), annual_years_vector(end)]);
title(ax1a, '(a) 平均法定退休年龄', 'FontName', 'SimSun', 'FontSize', 12);
ylabel(ax1a, '物理年龄 (岁)', 'FontName', 'SimSun');
xlabel(ax1a, '年份', 'FontName', 'SimSun');
legend(ax1a, 'show', 'Location', 'southeast', 'FontName', 'SimSun');

% -- Panel (b): 老年抚养比路径 --
ax1b = nexttile;
hold(ax1b, 'on');
for i = 1:length(scenarios)
    s = scenarios{i};
    dependency_ratio_path = zeros(1, cS_plot.T_sim);
    for t = 1:cS_plot.T_sim
        aR_t = results_all.(s).aR_new_path(t); % 获取当期退休年龄索引
        working_pop_t = sum(cS_plot.Z_path_raw(1:aR_t, t));
        retired_pop_t = sum(cS_plot.Z_path_raw(aR_t+1:end, t));
        if working_pop_t > 1e-9
            dependency_ratio_path(t) = (retired_pop_t / working_pop_t) * 100;
        else
            dependency_ratio_path(t) = NaN;
        end
    end
    plot(ax1b, time_axis_model, dependency_ratio_path, 'LineWidth', 2.5, 'Color', colors(s), 'LineStyle', line_styles_map(s));
end
hold(ax1b, 'off');
grid(ax1b, 'on'); box(ax1b, 'on');
xlim(ax1b, [time_axis_annual(1), time_axis_annual(end)]);
title(ax1b, '(b) 老年抚养比', 'FontName', 'SimSun', 'FontSize', 12);
ylabel(ax1b, '退休人口 / 劳动人口 (%)', 'FontName', 'SimSun');
xlabel(ax1b, '年份', 'FontName', 'SimSun');

% -- Panel (c): PAYG年度收支缺口/GDP (年度插值) --
ax1c = nexttile;
hold(ax1c, 'on');

for i = 1:length(scenarios)
    s = scenarios{i};
    deficit_to_gdp_annual = interp1(time_axis_model, results_all.(s).Deficit_to_GDP_path, time_axis_annual, 'pchip');
    plot(ax1c, time_axis_annual, deficit_to_gdp_annual*100, 'LineWidth', 2.5, 'Color', colors(s), 'LineStyle', line_styles_map(s));
end
yline(ax1c, 0, 'k--', 'LineWidth', 1, 'HandleVisibility', 'off');
hold(ax1c, 'off');
grid(ax1c, 'on'); box(ax1c, 'on');
xlim(ax1c, [time_axis_annual(1), time_axis_annual(end)]);
title(ax1c, '(c) 现收现付制年度收支缺口', 'FontName', 'SimSun', 'FontSize', 12);
ylabel(ax1c, '缺口占GDP比重 (%)', 'FontName', 'SimSun');
xlabel(ax1c, '年份', 'FontName', 'SimSun');

fprintf('   正在保存政策与缺口图...\n');
exportgraphics(fig1, output_fig_policy_deficit, 'Resolution', 300);


% --- 图2: 福利影响分析 ---
fig2 = figure('Name', '福利影响分析', 'Position', [461 646 637 276]);
tcl2 = tiledlayout(1, 2, 'Padding', 'compact', 'TileSpacing', 'compact');

cS_base = results_all.(base_scenario_for_welfare).data_tpi.cS;
T_max_comp_plot = size(cv_results.(reform_scenarios_for_welfare{1}), 2);
time_axis_welfare = cS_base.start_year : cS_base.time_Step : (cS_base.start_year + (T_max_comp_plot-1)*cS_base.time_Step);
time_mask_welfare = time_axis_welfare <= year_max;
time_axis_plot_welfare = time_axis_welfare(time_mask_welfare);

% --- 灰度绘图风格 ---
style_mid1 = {'-o', 'Color', [0.0 0.0 0.0], 'LineWidth', 2, 'MarkerSize', 4, 'MarkerFaceColor', [0.2 0.2 0.2]};
style_mid2 = {'--s', 'Color', [0.4 0.4 0.4], 'LineWidth', 2, 'MarkerSize', 4};
style_preretire = {':d', 'Color', [0.7 0.7 0.7], 'LineWidth', 2, 'MarkerSize', 4};
styles_age = {style_mid1, style_mid2, style_preretire};

% -- Panel (a): Moderate vs. Mild --
ax2a = nexttile;
hold(ax2a, 'on');
cv_data_mod = cv_results.moderate;
for i_age_grp = 1:num_age_groups_fig
    plot_data = cv_data_mod(i_age_grp, time_mask_welfare) * 100;
    plot(ax2a, time_axis_plot_welfare, plot_data, styles_age{i_age_grp}{:}, 'DisplayName', age_groups_for_figure{i_age_grp, 1});
end
yline(ax2a, 0, 'k--', 'LineWidth', 1.5, 'HandleVisibility', 'off');
hold(ax2a, 'off');
grid(ax2a, 'on'); box(ax2a, 'on');
xlim(ax2a, [time_axis_plot_welfare(1), time_axis_plot_welfare(end)]);
title(ax2a, '(a) 温和方案 vs. 平缓方案', 'FontName', 'SimSun', 'FontSize', 12);
ylabel(ax2a, ['相对于平缓方案的福利变化 (%)'], 'FontName', 'SimSun');
xlabel(ax2a, '年份', 'FontName', 'SimSun');
legend(ax2a, 'show', 'Location', 'best', 'FontName', 'SimSun');
ylim([-8,8])


% -- Panel (b): Aggressive vs. Mild --
ax2b = nexttile;
hold(ax2b, 'on');
cv_data_agg = cv_results.aggressive;
for i_age_grp = 1:num_age_groups_fig
    plot_data = cv_data_agg(i_age_grp, time_mask_welfare) * 100;
    plot(ax2b, time_axis_plot_welfare, plot_data, styles_age{i_age_grp}{:}, 'DisplayName', age_groups_for_figure{i_age_grp, 1});
end
yline(ax2b, 0, 'k--', 'LineWidth', 1.5, 'HandleVisibility', 'off');
hold(ax2b, 'off');
grid(ax2b, 'on'); box(ax2b, 'on');
xlim(ax2b, [time_axis_plot_welfare(1), time_axis_plot_welfare(end)]);
title(ax2b, '(b) 激进方案 vs. 平缓方案', 'FontName', 'SimSun', 'FontSize', 12);
xlabel(ax2b, '年份', 'FontName', 'SimSun');
ylim([-8,8])

fprintf('   正在保存福利影响图...\n');
exportgraphics(fig2, output_fig_welfare, 'Resolution', 300);

fprintf('\n--- 脚本执行完毕 ---\n');


%% --- [本地辅助函数] ---

function pathS_h = get_pathS_for_backward(results, iter_results, h, cS, ss0)
    % =====================================================================
    % == 函数: get_pathS_for_backward (v2.0 - 最终稳健版)
    % == 目的: 为指定的家庭类型 h，打包后向迭代所需的所有时间路径。
    % =====================================================================
    pathS_h = struct();
    pathS_h.r_path = iter_results.r_path_iter;
    w_hat_path_converged = iter_results.w_hat_path_iter;
    pathS_h.w_hat_path = w_hat_path_converged;
    pathS_h.tr_per_hh_hat_path = iter_results.TR_hat_pc_path_iter;
    w_hat_path_for_b = [ss0.w_hat, w_hat_path_converged(1:end-1)];
    b_hat_formula_path_h_all = zeros(cS.nTypes, cS.T_sim);
    for t = 1:cS.T_sim
        b_hat_formula_path_h_all(:, t) = SS.calculate_formula_benefits(w_hat_path_for_b(t), cS);
    end
    adj_path_converged = iter_results.adj_path_iter;
    pathS_h.b_hat_path = adj_path_converged .* b_hat_formula_path_h_all(h, :);
    A_path_ext = [cS.A_path, cS.A_path(end) * (1 + ((1+cS.g_A_ss)^cS.time_Step-1))];
    g_A_path = A_path_ext(2:end)./A_path_ext(1:end-1)-1;
    pathS_h.g_A_path = g_A_path(1:cS.T_sim);
    pathS_h.theta_path = cS.theta_path_h(h, :);
end


function cS_h = get_cs_for_type(cS, h)
% =====================================================================
% == 函数: get_cs_for_type (本地辅助函数)
% == 目的: 为指定的家庭类型 h，准备一个专属的 cS 参数包。
% =====================================================================
cS_h = cS;
cS_h.ageEffV_new = cS.ageEffV_new_h(:, h);
end

function [Total_Contributions_Level, Total_Formula_Benefits_Level] = recalculate_payg_flows_from_tpi_logic(w_hat_path, Dist_path_h, cS, paramSF, ss0)
    % =========================================================================
    % == 函数: recalculate_payg_flows_from_tpi_logic
    % == 目的: 精确复现 TPI 中的养老金收支计算逻辑。
    % =========================================================================
    T = cS.T_sim;
    nH = cS.nTypes;
    total_pension_pot_hat_path = zeros(1, T);
    total_formula_benefits_hat_path = zeros(1, T);
    w_hat_path_for_formula = [ss0.w_hat, w_hat_path(1:T-1)];
    b_hat_formula_path_h = zeros(nH, T);
    for t = 1:T
        b_hat_formula_path_h(:, t) = SS.calculate_formula_benefits(w_hat_path_for_formula(t), cS);
    end
    for t = 1:T
        total_contrib_t = 0;
        total_formula_benefit_t = 0;
        aR_t = cS.aR_new_path(t);
        for h = 1:nH
            L_path_supply_h_t = get_labor_supply_for_type_local(h, {Dist_path_h{h}(:,:,:,:,t)}, cS, paramSF, aR_t);
            total_contrib_t = total_contrib_t + cS.theta_path_h(h,t) * w_hat_path(t) * L_path_supply_h_t;
            mass_retirees_path_h_t = sum(cS.Z_path_raw((aR_t + 1):end, t)) * cS.type_weights(h);
            total_formula_benefit_t = total_formula_benefit_t + b_hat_formula_path_h(h, t) * mass_retirees_path_h_t;
        end
        total_pension_pot_hat_path(t) = total_contrib_t;
        total_formula_benefits_hat_path(t) = total_formula_benefit_t;
    end
    Total_Contributions_Level = total_pension_pot_hat_path .* cS.A_path;
    Total_Formula_Benefits_Level = total_formula_benefits_hat_path .* cS.A_path;
end

function L_path_h = get_labor_supply_for_type_local(h, Dist_path_h, cS, paramS, aR_t_period)
% =========================================================================
% == 函数: get_labor_supply_for_type_local
% == 目的: 精确复现 TPI.get_labor_supply_path_for_type 的逻辑。
% =========================================================================
T = size(Dist_path_h{1}, 5);
L_path_h = zeros(1, T);
cS_h = cS;
cS_h.ageEffV_new = cS.ageEffV_new_h(:,h);
dist_path_for_type_h = Dist_path_h{1};
for t = 1:T
    L_t_h = 0;
    dist_t_abs_h = dist_path_for_type_h(:,:,:,:,t);
    if length(aR_t_period) > 1, aR_now = aR_t_period(t); else, aR_now = aR_t_period; end
    for ia = 1:aR_now
        mass_ia_abs_h = dist_t_abs_h(:,:,:,ia);
        if sum(mass_ia_abs_h(:)) < 1e-30, continue; end
        le_grid_slice = reshape(paramS.leGridV, [1, 1, cS.nw_expanded]);
        labor_supply_slice = cS_h.ageEffV_new(ia) .* le_grid_slice;
        L_t_h = L_t_h + sum(labor_supply_slice .* mass_ia_abs_h, 'all');
    end
    L_path_h(t) = L_t_h;
end
end

function Val_path_welfare = recalculate_vpath_with_labor_disutility(Val_path, Pol_path, cS, paramS, labor_disutil_param)
    % =====================================================================
    % == 函数: recalculate_vpath_with_labor_disutility
    % == 目的: 重新计算包含劳动负效用的“福利衡量用”价值函数路径。
    % =====================================================================
    T = cS.T_sim;
    Val_path_welfare = zeros(size(Val_path));
    Val_path_welfare(:,:,:,:,T) = Val_path(:,:,:,:,T);
    A_path_ext = [cS.A_path, cS.A_path(end) * (1 + ((1+cS.g_A_ss)^cS.time_Step-1))];
    g_A_path = A_path_ext(2:end)./A_path_ext(1:end-1)-1;

    for t = (T-1) : -1 : 1
        valS_t_welfare = -Inf(cS.nk, cS.nkpps, cS.nw_expanded, cS.aD_new);
        Vprime_tp1_welfare = Val_path_welfare(:,:,:,:,t+1);
        g_A_period = g_A_path(t);
        growth_factor_bgp = (1 + g_A_period);
        discount_factor_V_prime = cS.beta * growth_factor_bgp^(1 - cS.sigma);
        pol_t_struct_array = Pol_path{t}; 

        for a_idx = cS.aD_new : -1 : 1
            pol_age_t = pol_t_struct_array(a_idx);
            c_mat = pol_age_t.c;
            if isempty(c_mat), continue; end
            util_c_mat = (c_mat.^(1-cS.sigma))./(1-cS.sigma);
            labor_supply_mat = pol_age_t.l;
            nu = 1.0;
            psi = labor_disutil_param;
            labor_disutil_mat = psi * (labor_supply_mat.^(1+nu))./(1+nu);
            ev_on_policy_welfare = zeros(cS.nk, cS.nkpps, cS.nw_expanded);
            if a_idx < cS.aD_new
                vPrime_next_age_welfare = Vprime_tp1_welfare(:,:,:,a_idx+1);
                k_prime_mat = pol_age_t.k_prime;
                kpps_prime_mat = pol_age_t.kpps_prime;
                trans_mats = paramS.TrProbM_by_age;
                vPrime_interpolants_welfare = cell(cS.nw_expanded, 1);
                for ie_next = 1:cS.nw_expanded
                    if cS.nkpps > 1 && cS.pps_active
                         vPrime_interpolants_welfare{ie_next} = griddedInterpolant({cS.kGridV, cS.kppsGridV}, vPrime_next_age_welfare(:, :, ie_next), 'pchip', 'pchip');
                    else
                         vPrime_interpolants_welfare{ie_next} = griddedInterpolant(cS.kGridV, squeeze(vPrime_next_age_welfare(:, :, ie_next)), 'pchip', 'pchip');
                    end
                end
                for ie = 1:cS.nw_expanded
                    trans_prob_row = trans_mats{a_idx}(ie, :);
                    ev_slice = zeros(cS.nk, cS.nkpps);
                    for ie_next = 1:cS.nw_expanded
                        if trans_prob_row(ie_next) > 1e-9
                           if cS.nkpps > 1 && cS.pps_active
                               v_interp = vPrime_interpolants_welfare{ie_next}(k_prime_mat(:,:,ie), kpps_prime_mat(:,:,ie));
                           else
                               v_interp = vPrime_interpolants_welfare{ie_next}(k_prime_mat(:,1,ie));
                           end
                           ev_slice = ev_slice + trans_prob_row(ie_next) * v_interp;
                        end
                    end
                    ev_on_policy_welfare(:,:,ie) = ev_slice;
                end
            end
            survival_rate = cS.s_pathV(a_idx, t);
            k_prime_for_bequest = pol_age_t.k_prime;
            if cS.pps_active, k_prime_for_bequest = k_prime_for_bequest + pol_age_t.kpps_prime; end
            util_bequest_mat = utils.bequest_utility(k_prime_for_bequest, cS);
            Future_V_welfare_discounted = discount_factor_V_prime * (survival_rate * ev_on_policy_welfare + (1 - survival_rate) * util_bequest_mat);
            valS_t_welfare(:,:,:,a_idx) = util_c_mat - labor_disutil_mat + Future_V_welfare_discounted;
        end
        Val_path_welfare(:,:,:,:,t) = valS_t_welfare;
    end
end

## 技术增长

### 表 4.3--不同经济增长情景下的宏观经济与养老金可持续性关键指标

### 表 4.4--{不同经济增长情景对各类型家庭的福利影响 (相对于基准情景的补偿变化 CV, \%)

### 图 4.7--TFP年化增长率路径

In [ ]:
% =========================================================================
% == SCRIPT: analyze_tfp_scenarios.m
% == 版本: [v1.0 - TFP情景分析与adj框架]
% ==
% == 目的:
% ==   1. 生成 表 4.3: 不同经济增长情景下的宏观经济与养老金可持续性
% ==      关键指标，核心为内生调整因子(adj)。
% ==   2. 生成 表 4.4: 不同经济增长情景对各类型家庭的福利影响
% ==      (CV, %)，以基准情景为比较对象。
% ==
% == 前置条件:
% ==   - 已成功运行 main_run_trans.m 并生成以下文件:
% ==     - TRANS/TPI_results_het_nopps_TFP_baseline.mat
% ==     - TRANS/TPI_results_het_nopps_TFP_optimistic.mat
% ==     - TRANS/TPI_results_het_nopps_TFP_pessimistic.mat
% =========================================================================

clear; close all; clc;
addpath(pwd);
fprintf('=== TFP冲击情景对比分析脚本 (adj框架 v1.0) ===\n\n');

%% --- 1. 用户设定与文件定义 ---
fprintf('--- 1. 用户设定与文件定义 ---\n');

% --- 分析设定 ---
scenarios = {'pessimistic', 'baseline', 'optimistic'};
base_scenario = 'baseline'; % 福利分析的基准情景

% --- 文件路径模板 ---
file_tpi_pattern = 'TRANS/TPI_results_het_nopps_TFP_%s.mat';

% --- 输出路径 ---
output_dir_tex = 'tex/tab/';
if ~exist(output_dir_tex, 'dir'), mkdir(output_dir_tex); end
output_tex_table_macro = fullfile(output_dir_tex, 'tab4.3_macro_sustainability.tex');
output_tex_table_welfare = fullfile(output_dir_tex, 'tab4.4_welfare_analysis.tex');

fprintf('   福利分析基准方案: %s\n', base_scenario);
fprintf('   LaTeX 表格将输出至: %s\n', output_dir_tex);


%% --- 2. [数据加载] 加载所有情景的计算结果 ---
fprintf('\n--- 2. [数据加载] 循环加载各方案结果 ---\n');

results_all = struct(); % 用于存储所有方案的核心计算结果

for i = 1:length(scenarios)
    scenario_name = scenarios{i};
    fprintf('   正在加载方案: [%s] ...\n', upper(scenario_name));

    % --- 加载数据 ---
    file_tpi = sprintf(file_tpi_pattern, scenario_name);
    if ~exist(file_tpi, 'file')
        error('找不到文件: %s，请确认main_run_trans.m已针对该情景运行。', file_tpi); 
    end

    results_all.(scenario_name) = load(file_tpi);
    fprintf('   ✅ 已加载: %s\n', file_tpi);
end
fprintf('--- 所有情景数据加载完毕 ---\n');




%% --- 3. [生成表4.3] 宏观经济与养老金可持续性 ---
fprintf('\n--- 3. [生成表4.3] 宏观经济与养老金可持续性 ---\n');

% --- [核心修正] 定义评估年份和指标 (增加 K/AL) ---
assessment_years_macro = [2030, 2050];
macro_indicators = {'r', 'w_hat', 'K_AL', 'adj'};
indicator_labels = {'年化真实利率 (r, \%)', '有效工资率 (w, 指数)', '资本-效率劳动比 (K/AL)', '均衡调整因子 (adj)'};
section_labels = {'A. 宏观经济与要素价格', 'B. PAYG养老金体系可持续性'};

% --- 打开文件并写入LaTeX表头 ---
fileID = fopen(output_tex_table_macro, 'w', 'n', 'UTF-8');
fprintf(fileID, '%% =======================================================\n');
fprintf(fileID, '%%  此文件由 analyze_tfp_scenarios.m (v1.4, K/AL版) 自动生成\n');
fprintf(fileID, '%% =======================================================\n');
fprintf(fileID, '\\begin{table}[h!]\n');
fprintf(fileID, '\\centering\n');
fprintf(fileID, '\\caption{不同经济增长情景下的宏观经济与养老金可持续性关键指标}\n');
fprintf(fileID, '\\label{tab4.3:macro_sustainability_adj}\n');
fprintf(fileID, '\\begin{threeparttable}\n');
fprintf(fileID, '\\begin{tabular}{llccc}\n');
fprintf(fileID, '\\toprule\n');
fprintf(fileID, '指标 & 评估年份 & 悲观情景 & 基准情景 & 乐观情景 \\\\\n');
fprintf(fileID, '\\midrule\n');

% --- 循环提取数据并写入表格 ---
for i_sec = 1:length(section_labels)
    fprintf(fileID, '\\multicolumn{5}{l}{\\textbf{%s}} \\\\\n', section_labels{i_sec});
    
    current_indicators = {};
    if i_sec == 1, current_indicators = {'r', 'w_hat', 'K_AL'}; end
    if i_sec == 2, current_indicators = {'adj'}; end

    for i_ind = 1:length(current_indicators)
        indicator = current_indicators{i_ind};
        label_idx = find(strcmp(macro_indicators, indicator));
        
        fprintf(fileID, '%s', indicator_labels{label_idx});

        % --- [核心修正] 写入期初均衡数据 (从各自的 ss0 提取) ---
        fprintf(fileID, ' & 期初均衡');
        for i_scen = 1:length(scenarios)
            scen_name = scenarios{i_scen};
            data = results_all.(scen_name);
            ss0 = data.ss0;
            cS = data.cS;
            
            switch indicator
                case 'r'
                    value = ((1 + ss0.r_mkt)^(1/cS.time_Step) - 1) * 100;
                    fprintf(fileID, ' & %.3f\\%%', value);
                case 'w_hat'
                    value = ss0.w_hat;
                    fprintf(fileID, ' & %.3f', value);
                case 'K_AL'
                    % K_AL_ss = K_p_hat_ss / (A_ss * L_hat_ss) -> A_ss=1, K_p_hat / L_hat
                    value = ss0.K_private_hat / ss0.L_hat;
                    fprintf(fileID, ' & %.3f', value);
                case 'adj'
                    value = ss0.adj;
                    fprintf(fileID, ' & %.3f', value);
            end
        end
        fprintf(fileID, ' \\\\\n');
        
        % --- 写入转轨期数据 ---
        for i_yr = 1:length(assessment_years_macro)
            year = assessment_years_macro(i_yr);
            fprintf(fileID, ' & %d', year); % 评估年份列

            for i_scen = 1:length(scenarios)
                scen_name = scenarios{i_scen};
                data = results_all.(scen_name);
                t_idx = round((year - data.cS.start_year) / data.cS.time_Step) + 1;
                
                if t_idx > 0 && t_idx <= data.cS.T_sim
                    switch indicator
                        case 'r'
                            r_period = data.results.iter_results.r_path_iter(t_idx);
                            value = ((1 + r_period)^(1/data.cS.time_Step) - 1) * 100;
                            fprintf(fileID, ' & %.3f\\%%', value);
                        case 'w_hat'
                            value = data.results.iter_results.w_hat_path_iter(t_idx);
                            fprintf(fileID, ' & %.3f', value);
                        case 'K_AL'
                            aggr_supply = aggregates.get_path_aggregates(data.final_Dist_path_h, data.final_Pol_path_h, data.cS, data.results.ss_data.paramSF);
                            K_p_hat_t = aggr_supply.K_p_hat_total_path(t_idx);
                            L_hat_t = aggr_supply.L_path(t_idx);
                            % K/AL = K_hat*A*N / (A*L_hat*N) = K_hat / L_hat
                            value = K_p_hat_t / L_hat_t;
                            fprintf(fileID, ' & %.3f', value);
                        case 'adj'
                            value = data.results.iter_results.adj_path_iter(t_idx);
                            fprintf(fileID, ' & %.3f', value);
                    end
                else
                    fprintf(fileID, ' & -');
                end
            end
            fprintf(fileID, ' \\\\\n');
        end
        
        % --- 写入终期稳态数据 ---
        fprintf(fileID, ' & 终期稳态');
        for i_scen = 1:length(scenarios)
            scen_name = scenarios{i_scen};
            data = results_all.(scen_name);
            ssF = data.ssF;
            cS = data.cS;

            switch indicator
                case 'r'
                    value = ((1 + ssF.r_mkt)^(1/cS.time_Step) - 1) * 100;
                    fprintf(fileID, ' & %.3f\\%%', value);
                case 'w_hat'
                    value = ssF.w_hat;
                    fprintf(fileID, ' & %.3f', value);
                case 'K_AL'
                    value = ssF.K_private_hat / ssF.L_hat;
                    fprintf(fileID, ' & %.3f', value);
                case 'adj'
                    value = ssF.adj;
                    fprintf(fileID, ' & %.3f', value);
            end
        end
        fprintf(fileID, ' \\\\\n');
        
        if (i_sec == 1 && i_ind < length(current_indicators)) || (i_sec == 2 && i_ind < length(current_indicators))
            fprintf(fileID, '\\addlinespace\n');
        end
        
    end
end

% --- 写入表尾和注释 ---
fprintf(fileID, '\\bottomrule\n');
fprintf(fileID, '\\end{tabular}\n');
fprintf(fileID, '\\begin{tablenotes}[para,flushleft]\n');
fprintf(fileID, '  \\item 注： 资本-效率劳动比(K/AL)是私人资本存量与经技术调整后的总劳动供给之比。均衡调整因子(adj)是在保持缴费率固定的前提下，为保证PAYG体系收支平衡所必需的福利调整系数。adj=1.0表示福利可100\\%%兑现。有效工资率(w)是剔除技术进步影响的工资指数。所有情景均基于无个人养老金账户(nopps)的设定。\n');
fprintf(fileID, '\\end{tablenotes}\n');
fprintf(fileID, '\\end{threeparttable}\n');
fprintf(fileID, '\\end{table}\n');
fclose(fileID);
fprintf('   ✅ 成功生成 LaTeX 表格文件: %s\n', output_tex_table_macro);

%% --- 4. [福利分析] 为各情景重构价值函数路径 ---
fprintf('\n--- 4. [福利分析] 重构价值函数路径 ---\n');

for i_scen = 1:length(scenarios)
    scen_name = scenarios{i_scen};
    fprintf('   正在为方案 [%s] 运行反向迭代...\n', upper(scen_name));
    
    data = results_all.(scen_name);
    cS = data.cS;
    paramSF = data.results.ss_data.paramSF;
    valF_h = data.results.ss_data.valF_h;
    polF_h = data.results.ss_data.polF_h;
    iter_results = data.results.iter_results;
    
    Val_path_h = cell(cS.nTypes, 1);
    
    for h = 1:cS.nTypes
        % 获取向后迭代所需的所有路径
        pathS_h = get_pathS_for_backward(iter_results, h, cS, data.ss0);
        
        % 获取该类型特定的参数
        cS_h = get_cs_for_type(cS, h);
        
        % 运行后向迭代
        [~, Val_path_h{h}] = household.backward_hh(pathS_h, cS_h, paramSF, valF_h{h}, polF_h{h});
    end
    
    % 将计算得到的价值函数存回主结构体
    results_all.(scen_name).Val_path_h = Val_path_h;
    fprintf('   ✅ 方案 [%s] 价值函数重构完成。\n', upper(scen_name));
end
fprintf('--- 所有情景价值函数路径重构完毕 ---\n');


%% --- 5. [生成表4.4] 计算并生成福利分析(CV)表格 ---
fprintf('\n--- 5. [生成表4.4] 计算并生成福利分析(CV)表格 ---\n');

% --- 定义分析维度 ---
assessment_years_welfare = [2030, 2050, 2070, 2100];
% [核心修正] 明确定义改革情景的顺序，以匹配LaTex表头
reform_scenarios = {'optimistic', 'pessimistic'}; 
type_labels = {'高收入', '中低收入'};
sector_labels = {'A. 城镇职工', 'B. 城乡居民'};

% --- 获取基准情景的数据 ---
data_base = results_all.(base_scenario);
cS_base = data_base.cS;
sigma = cS_base.sigma;
T_sim_base = cS_base.T_sim;

% --- [核心修正] 在 cS_base 定义后，直接计算年龄组索引，避免使用 eval ---
age_groups = {
    '青年 (25-29岁)', round((25 - cS_base.age1_orig)/cS_base.time_Step) + 1;
    '中年 (45-49岁)', round((45 - cS_base.age1_orig)/cS_base.time_Step) + 1;
    '临退 (60-64岁)', cS_base.aR_new_path(1) % 使用初始退休年龄作为代表
    '老年 (70-74岁)', round((70 - cS_base.age1_orig)/cS_base.time_Step) + 1;
};

% --- 打开文件并写入LaTeX表头 ---
fileID_welfare = fopen(output_tex_table_welfare, 'w', 'n', 'UTF-8');
fprintf(fileID_welfare, '%% =======================================================\n');
fprintf(fileID_welfare, '%%  此文件由 analyze_tfp_scenarios.m (v1.2, 修正版) 自动生成\n');
fprintf(fileID_welfare, '%% =======================================================\n');
fprintf(fileID_welfare, '\\begin{table}[h!]\n');
fprintf(fileID_welfare, '\\centering\n');
fprintf(fileID_welfare, '\\caption{不同经济增长情景对各类型家庭的福利影响 (\\%%)}\n');
fprintf(fileID_welfare, '\\label{tab4.4:welfare_impact_tfp}\n');
fprintf(fileID_welfare, '\\begin{threeparttable}\n');

% [核心修正] 动态、安全地构建并写入表头
header_cols = 'l';
header_line_1 = {'家庭类型与年龄组'};
header_line_2 = {''};
cmidrule_parts = {};
num_data_cols = 0;

for i_yr = 1:length(assessment_years_welfare)
    header_cols = [header_cols, 'rr'];
    header_line_1{end+1} = sprintf('\\multicolumn{2}{c}{%d}', assessment_years_welfare(i_yr));
    header_line_2{end+1} = '乐观';
    header_line_2{end+1} = '悲观';
    cmidrule_parts{end+1} = sprintf('\\cmidrule(lr){%d-%d}', num_data_cols + 2, num_data_cols + 3);
    num_data_cols = num_data_cols + 2;
end

final_header_1 = strjoin(header_line_1, ' & ');
final_header_2 = strjoin(header_line_2, ' & ');
final_cmidrule = strjoin(cmidrule_parts, ' ');

fprintf(fileID_welfare, '\\begin{tabular}{%s}\n', header_cols);
fprintf(fileID_welfare, '\\toprule\n');
fprintf(fileID_welfare, '%s \\\\\n', final_header_1);
fprintf(fileID_welfare, '%s\n', final_cmidrule);
fprintf(fileID_welfare, '%s \\\\\n', final_header_2);
fprintf(fileID_welfare, '\\midrule\n');

% --- [核心逻辑修正] 核心CV计算与写入 ---
for i_sector = 1:length(sector_labels)
    fprintf(fileID_welfare, '\\multicolumn{%d}{l}{\\textbf{%s}} \\\\\n', num_data_cols + 1, sector_labels{i_sector});
    for i_type = 1:length(type_labels)
        h_idx = (i_sector - 1) * 2 + i_type;
        
        % 打印类型标签，并填充该行的空数据列
        fprintf(fileID_welfare, '    \\hspace{1em}%s', type_labels{i_type});
        fprintf(fileID_welfare, repmat(' & ', 1, num_data_cols));
        fprintf(fileID_welfare, ' \\\\\n');

        for i_age = 1:size(age_groups, 1)
            age_label = age_groups{i_age, 1};
            a_idx = age_groups{i_age, 2};
            
            fprintf(fileID_welfare, '       \\hspace{2em}%s', age_label);

            for i_yr = 1:length(assessment_years_welfare)
                year = assessment_years_welfare(i_yr);
                t_idx = round((year - cS_base.start_year) / cS_base.time_Step) + 1;

                if t_idx > T_sim_base
                    fprintf(fileID_welfare, ' & - & -');
                    continue;
                end
                
                % 按照 "乐观 vs. 基准", "悲观 vs. 基准" 的顺序循环
                for i_ref = 1:length(reform_scenarios)
                    reform_name = reform_scenarios{i_ref};
                    data_reform = results_all.(reform_name);
                    
                    dist_base_slice = data_base.final_Dist_path_h{h_idx}(:, :, :, a_idx, t_idx);
                    val_base_slice  = data_base.Val_path_h{h_idx}(:, :, :, a_idx, t_idx);
                    val_reform_slice= data_reform.Val_path_h{h_idx}(:, :, :, a_idx, t_idx);
                    
                    mass_slice = sum(dist_base_slice, 'all');
                    
                    cv = NaN;
                    if mass_slice > 1e-12
                        EV_base = sum(val_base_slice .* dist_base_slice, 'all') / mass_slice;
                        EV_reform = sum(val_reform_slice .* dist_base_slice, 'all') / mass_slice;
                        
                        if EV_base < -1e-9 && EV_reform < -1e-9
                            cv = (EV_reform / EV_base)^(1 / (1 - sigma)) - 1;
                        end
                    end
                    fprintf(fileID_welfare, ' & %s', format_cv(cv));
                end % reform scenarios loop
            end % year loop
            fprintf(fileID_welfare, ' \\\\\n');
        end % age group loop
    end % type loop
    if i_sector == 1
         fprintf(fileID_welfare, '\\midrule\n');
    end
end % sector loop

% --- 计算并写入总体平均 ---
fprintf(fileID_welfare, '\\midrule\n');
fprintf(fileID_welfare, '\\multicolumn{%d}{l}{\\textbf{C. 总体平均}} \\\\\n', num_data_cols + 1);
fprintf(fileID_welfare, '    人口加权平均 CV');

for i_yr = 1:length(assessment_years_welfare)
    year = assessment_years_welfare(i_yr);
    t_idx = round((year - cS_base.start_year) / cS_base.time_Step) + 1;
    
    if t_idx > T_sim_base
        fprintf(fileID_welfare, ' & - & -');
        continue;
    end
    
    total_pop_t = sum(cS_base.Z_path_raw(:, t_idx), 'all');
    
    for i_ref = 1:length(reform_scenarios)
        reform_name = reform_scenarios{i_ref};
        data_reform = results_all.(reform_name);
        
        total_weighted_ev_base = 0;
        total_weighted_ev_reform = 0;
        
        for h_idx = 1:cS_base.nTypes
            for a_idx = 1:cS_base.aD_new
                dist_base_slice = data_base.final_Dist_path_h{h_idx}(:, :, :, a_idx, t_idx);
                val_base_slice  = data_base.Val_path_h{h_idx}(:, :, :, a_idx, t_idx);
                val_reform_slice= data_reform.Val_path_h{h_idx}(:, :, :, a_idx, t_idx);
                
                total_weighted_ev_base   = total_weighted_ev_base   + sum(val_base_slice .* dist_base_slice, 'all');
                total_weighted_ev_reform = total_weighted_ev_reform + sum(val_reform_slice .* dist_base_slice, 'all');
            end
        end
        
        EV_base_total_avg = total_weighted_ev_base / total_pop_t;
        EV_reform_total_avg = total_weighted_ev_reform / total_pop_t;
        
        cv_total = NaN;
        if EV_base_total_avg < -1e-9 && EV_reform_total_avg < -1e-9
             cv_total = (EV_reform_total_avg / EV_base_total_avg)^(1 / (1 - sigma)) - 1;
        end
        fprintf(fileID_welfare, ' & %s', format_cv(cv_total));
    end
end
fprintf(fileID_welfare, ' \\\\\n');


% --- 写入表尾 ---
fprintf(fileID_welfare, '\\bottomrule\n');
fprintf(fileID_welfare, '\\end{tabular}\n');
fprintf(fileID_welfare, '\\begin{tablenotes}[para,flushleft]\n');
% [核心修正] 更新注释以匹配简化的表头
fprintf(fileID_welfare, '  \\item 注： 表中数值为补偿变化(CV)，衡量“乐观”或“悲观”情景相对于“基准”情景的福利差异。正值代表福利改善。例如，+0.8\\%%表示该群体愿意放弃其在基准情景下剩余生命周期消费的0.8\\%%，以换取进入乐观增长情景。所有计算均基于固定缴费率和内生福利调整因子(adj)的框架。\n');
fprintf(fileID_welfare, '\\end{tablenotes}\n');
fprintf(fileID_welfare, '\\end{threeparttable}\n');
fprintf(fileID_welfare, '\\end{table}\n');
fclose(fileID_welfare);
fprintf('   ✅ 成功生成 LaTeX 表格文件: %s\n', output_tex_table_welfare);

fprintf('\n--- 脚本执行完毕 ---\n');

%% --- 7. [生成图4.7] 绘制三种情景的TFP增长率路径 ---
fprintf('\n--- 7. [生成图4.7] 绘制三种情景的TFP增长率路径 ---\n');

% --- 定义输出路径 ---
output_dir_fig = 'tex/fig/';
if ~exist(output_dir_fig, 'dir'), mkdir(output_dir_fig); end
output_fig_tfp = fullfile(output_dir_fig, 'fig4.7_tfp_path.png');

% --- 设定绘图参数 ---
% 从任一情景加载cS以获取时间轴参数
cS = results_all.baseline.cS; 
annual_years_vec = cS.start_year:cS.end_year;

% --- 设定灰度颜色和线型 ---
color_pessimistic = [0.65 0.65 0.65]; 
color_baseline = [0.0 0.0 0.0]; 
color_optimistic = [0.35 0.35 0.35];
colors = containers.Map({'pessimistic', 'baseline', 'optimistic'}, {color_pessimistic, color_baseline, color_optimistic});
line_styles = {':', '-', '--'}; 
line_styles_map = containers.Map({'pessimistic', 'baseline', 'optimistic'}, line_styles);
% [核心修正] 中文图例标签
legend_labels_map = containers.Map({'pessimistic', 'baseline', 'optimistic'}, {'悲观情景', '基准情景', '乐观情景'});

% --- [核心修正] 初始化图形，使用指定位置 ---
fig_tfp = figure('Name', 'TFP增长率路径情景', 'Position', [100 428 425 172]);
hold on;

% --- 循环生成并绘制路径 ---
for i = 1:length(scenarios)
    scen_name = scenarios{i};
    % 调用函数生成该情景的年度增长率路径
    [~, g_path_annual, ~] = utils.generate_tfp_path(cS, scen_name, false);
    
    plot(annual_years_vec, g_path_annual * 100, ...
        'Color', colors(scen_name), ...
        'LineStyle', line_styles_map(scen_name), ...
        'LineWidth', 2.5, ...
        'DisplayName', legend_labels_map(scen_name));
end

% --- [核心修正] 添加图表格式，移除 yline ---
xlabel('年份', 'FontName', 'SimSun');
ylabel('年化增长率 (%)', 'FontName', 'SimSun');
legend('show', 'Location', 'best', 'FontName', 'SimSun');
grid on;
box on;
xlim([cS.start_year, 2150]);

% --- 保存图像 ---
fprintf('   正在保存TFP路径图至: %s\n', output_fig_tfp);
try
    exportgraphics(fig_tfp, output_fig_tfp, 'Resolution', 300);
    fprintf('   ✅ 图像保存成功。\n');
catch ME
    fprintf('   ❌ 图像保存失败！错误信息: %s\n', ME.message);
end

%% --- [本地辅助函数] ---

function pathS_h = get_pathS_for_backward(iter_results, h, cS, ss0)
    % =====================================================================
    % == 函数: get_pathS_for_backward (本地辅助函数)
    % == 目的: 为指定的家庭类型 h，打包后向迭代所需的所有时间路径。
    % =====================================================================
    pathS_h = struct();
    pathS_h.r_path = iter_results.r_path_iter;
    w_hat_path_converged = iter_results.w_hat_path_iter;
    pathS_h.w_hat_path = w_hat_path_converged;
    pathS_h.tr_per_hh_hat_path = iter_results.TR_hat_pc_path_iter;
    
    % --- 重建养老金路径 (b_hat_path) ---
    % 养老金公式基于上一期的工资，所以需要构造一个错位的工资路径
    w_hat_path_for_b = [ss0.w_hat, w_hat_path_converged(1:end-1)];
    b_hat_formula_path_h_all = zeros(cS.nTypes, cS.T_sim);
    for t = 1:cS.T_sim
        % 调用SS命名空间下的静态函数来计算公式养老金
        b_hat_formula_path_h_all(:, t) = SS.calculate_formula_benefits(w_hat_path_for_b(t), cS);
    end
    adj_path_converged = iter_results.adj_path_iter;
    pathS_h.b_hat_path = adj_path_converged .* b_hat_formula_path_h_all(h, :);
    
    % --- 重建增长率和缴费率路径 ---
    A_path_ext = [cS.A_path, cS.A_path(end) * (1 + ((1+cS.g_A_ss)^cS.time_Step-1))];
    g_A_path = A_path_ext(2:end)./A_path_ext(1:end-1)-1;
    pathS_h.g_A_path = g_A_path(1:cS.T_sim);
    pathS_h.theta_path = cS.theta_path_h(h, :);
end


function cS_h = get_cs_for_type(cS, h)
    % =====================================================================
    % == 函数: get_cs_for_type (本地辅助函数)
    % == 目的: 为指定的家庭类型 h，准备一个专属的 cS 参数包。
    % =====================================================================
    cS_h = cS;
    cS_h.ageEffV_new = cS.ageEffV_new_h(:, h);
    % aR_new_path 已经是特定于此cS的，无需修改
end

function str_out = format_cv(cv_value)
    % 辅助函数: 将CV数值格式化为带正负号和百分号的字符串
    if isnan(cv_value)
        str_out = '-';
        return;
    end
    
    if cv_value >= 0
        sign_char = '+';
    else
        sign_char = ''; % 负号会自动显示
    end
    str_out = sprintf('%s%.1f\\%%', sign_char, cv_value * 100);
end

## 个人养老金

### 图4.8 PAYG主导转向PPS主导的路径分析

In [ ]:
% =========================================================================
% == SCRIPT: create_fig_structural_reform.m
% == 版本: [v1.2 - 尺寸与灰度修正版]
% ==
% == 目的:
% ==   1. 加载'structural_reform'情景下的转轨动态结果。
% ==   2. 对最终的分布路径进行后处理，聚合出分类型的PPS资本和总PAYG缴费。
% ==   3. 严格遵循'ref_for_graph.m'的风格，生成一个1x3的图，展示
% ==      从PAYG主导向PPS主导转变的关键宏观和政策路径。
% ==
% == v1.2 核心修改:
% ==   - [!!!] 将图形宽度严格设置为 14.99cm，以满足排版要求。
% ==   - [!!!] 将子图2 (PAYG) 的所有元素统一为灰度风格。
% =========================================================================

clear; close all; clc;
addpath(pwd);
fprintf('=== 结构性养老金改革(structural_reform)可视化脚本 (v1.2) ===\n\n');

%% --- 1. 用户设定与数据加载 ---
fprintf('--- 1. 设定与文件加载 ---\n');

% --- 输入文件 ---
input_file = 'TRANS/TPI_results_het_pps_structural_reform.mat';

% --- 输出路径 ---
output_dir = 'tex/fig/';
if ~exist(output_dir, 'dir'), mkdir(output_dir); end
output_filename = fullfile(output_dir, 'fig4.8_pps_structural_reform.png');

% --- 绘图设定 ---
plot_start_year = 2023;
plot_end_year = 2100;

% --- 加载数据 ---
fprintf('   正在加载文件: %s ...\n', input_file);
if ~exist(input_file, 'file'), error('找不到TPI结果文件: %s', input_file); end
data = load(input_file);
cS = data.cS;
results = data.results;
dist_path_h = data.final_Dist_path_h;
paramSF = data.results.ss_data.paramSF;
fprintf('   ✅ 数据加载完成。\n');


%% --- 2. [核心] 数据后处理与计算 ---
fprintf('\n--- 2. 正在进行后处理计算 ---\n');
T = cS.T_sim;
nH = cS.nTypes;
Y_path = results.Y_path;

% --- 2.1 聚合分类型的PPS资本路径 ---
fprintf('   2.1 聚合分类型PPS资本...\n');
[Kpps_hat_path_h, Kpps_path_h] = aggregate_pps_capital_by_type(dist_path_h, cS);
Kpps_to_GDP_path_h = Kpps_path_h ./ Y_path * 100; % 转换为百分比
fprintf('       ✅ 完成。\n');

% --- 2.2 聚合总PAYG缴费路径 ---
fprintf('   2.2 聚合总PAYG缴费...\n');
PAYG_Contrib_hat_path = aggregate_payg_contributions(dist_path_h, results.w_path, cS, paramSF);
PAYG_Contrib_path = PAYG_Contrib_hat_path .* cS.A_path;
PAYG_to_GDP_path = PAYG_Contrib_path ./ Y_path * 100; % 转换为百分比
fprintf('       ✅ 完成。\n');

% --- 2.3 计算宏观比率和价格路径 ---
fprintf('   2.3 计算宏观比率与价格路径...\n');
Kp_to_Y_path = results.K_p_path ./ Y_path;
C_to_Y_path = results.C_path ./ Y_path;
w_path = results.w_path;
r_annual_path = ((1 + results.r_path).^(1/cS.time_Step) - 1) * 100; % 年化利率
fprintf('       ✅ 完成。\n');

%% --- 3. [核心] 可视化 ---
fprintf('\n--- 3. 正在生成1x3图表 ---\n');

% --- 绘图风格设定 (遵循 ref_for_graph.m) ---
% [!!! 核心修改 !!!] 严格设置图形尺寸
fig = figure('Name', '结构性养老金改革路径分析');
fig.Units = 'centimeters';
fig.Position = [10 10 14.99 5.5]; % [left bottom width height]

tcl = tiledlayout(1, 3, 'Padding', 'tight', 'TileSpacing', 'compact');

time_axis = cS.start_year : cS.time_Step : (cS.start_year + (T-1)*cS.time_Step);
time_mask = (time_axis >= plot_start_year) & (time_axis <= plot_end_year);
time_axis_plot = time_axis(time_mask);

% --- 子图1: PPS 资本 与 缴费上限 ---
ax1 = nexttile;
hold(ax1, 'on');

bar_data = Kpps_to_GDP_path_h(:, time_mask)';
b1 = bar(ax1, time_axis_plot, bar_data, 'stacked');

colors_bar = [0.2 0.2 0.2; 0.45 0.45 0.45; 0.65 0.65 0.65; 0.85 0.85 0.85];
for i = 1:nH
    b1(i).FaceColor = colors_bar(i, :);
end

plot(ax1, time_axis_plot, cS.pps_fixed_path(time_mask) * 100, 'k-s', 'LineWidth', 0.3, 'MarkerFaceColor', 'k','MarkerSize',3);
ax1.YAxis.Color = 'k';

grid(ax1, 'on'); box(ax1, 'on');
xlim(ax1, [plot_start_year, plot_end_year]);
title(ax1, '（a）个人养老金路径（%）',  'FontSize', 9);
xlabel(ax1, '年份',  'FontSize', 9);
l1=legend(ax1, '高收城镇', '中低收城镇', '高收居民', '中低收居民', '缴费上限', 'Position',[0.0275 0.5513 0.1070 0.3654],...
    'IconColumnWidth',5,'Box','off','FontSize', 9);

% --- 子图2: PAYG 缴费 与 政策路径 ---
ax2 = nexttile;
hold(ax2, 'on');

% [!!! 核心修改 !!!] 统一为灰度图
p0=bar(ax2, time_axis_plot, PAYG_to_GDP_path(time_mask), 'FaceColor', [0.85 0.85 0.85], 'EdgeColor', 'none', 'DisplayName','PAYG/GDP');
p1 = plot(ax2, time_axis_plot, mean(cS.theta_path_h(:, time_mask),1) * 100, '--', 'Color', [0.0 0.0 0.0], 'LineWidth', 1, 'DisplayName', 'PAYG费率');
p2 = plot(ax2, time_axis_plot, results.adj_path(time_mask), ':', 'Color', [0.4 0.4 0.4], 'LineWidth', 1.5, 'DisplayName', '调整因子');

ax2.YAxis.Color = 'k';

grid(ax2, 'on'); box(ax2, 'on');
xlim(ax2, [plot_start_year, plot_end_year]);
title(ax2, '（b）PAYG路径（%）',  'FontSize', 9);
xlabel(ax2, '年份');
l2=legend(ax2, [p0, p1, p2], 'Position',[0.4267 0.7460 0.1764 0.1562],...
    'Box','off','IconColumnWidth',10,'FontSize', 9);


% --- 子图3: 关键宏观变量路径 ---
ax3 = nexttile;
hold(ax3, 'on');

yyaxis(ax3, 'left');
p_ky = plot(ax3, time_axis_plot, Kp_to_Y_path(time_mask), '-', 'Color', [0.2 0.2 0.2], 'LineWidth', 1.5, 'DisplayName', 'K/Y（左）');
p_cy = plot(ax3, time_axis_plot, C_to_Y_path(time_mask), '--', 'Color', [0.6 0.6 0.6], 'LineWidth', 1.5, 'DisplayName', 'C/Y（左）');
ax3.YAxis(1).Color = 'k';

yyaxis(ax3, 'right');
% p_w = plot(ax3, time_axis_plot, w_path(time_mask), '-.', 'Color', [0.4 0.4 0.4], 'LineWidth', 2, 'DisplayName', '工资 (右轴)');
p_r = plot(ax3, time_axis_plot, r_annual_path(time_mask), ':', 'Color', [0.0 0.0 0.0], 'LineWidth', 1.5, 'DisplayName', '利率（右,%）');
ax3.YAxis(2).Color = 'k';

grid(ax3, 'on'); box(ax3, 'on');
xlim(ax3, [plot_start_year, plot_end_year]);
title(ax3, '（c）宏观经济路径', 'FontSize', 9);
xlabel(ax3, '年份',  'FontSize', 9);
l3=legend(ax3, 'show', 'Position',[0.8355 0.3860 0.1035 0.2260],...
    'IconColumnWidth',5,'Box','off', 'FontSize', 8);


%% --- 4. 保存图表 ---
fprintf('\n--- 4. 正在保存图表 ---\n');
try
    exportgraphics(fig, output_filename, 'Resolution', 300);
    fprintf('   ✅ 成功保存图表至: %s\n', output_filename);
catch ME
    fprintf('   ❌ 保存失败: %s\n', ME.message);
end

fprintf('\n--- 脚本执行完毕 ---\n');


%% --- [本地辅助函数] ---

function [Kpps_hat_path_h, Kpps_path_h] = aggregate_pps_capital_by_type(dist_path_h, cS)
    % 聚合每个类型家庭的PPS资本存量路径 (hat量 和 level量)
    nH = cS.nTypes;
    T = cS.T_sim;
    Kpps_hat_path_h = zeros(nH, T);

    kpps_grid_full = repmat(reshape(cS.kppsGridV, [1, cS.nkpps]), [cS.nk, 1, cS.nw_expanded]);

    for t = 1:T
        for h = 1:nH
            dist_t_h = dist_path_h{h}(:,:,:,:,t);
            kpps_hat_t_h = 0;
            for ia = 1:cS.aD_new
                dist_slice = dist_t_h(:,:,:,ia);
                kpps_hat_t_h = kpps_hat_t_h + sum(kpps_grid_full .* dist_slice, 'all');
            end
            Kpps_hat_path_h(h, t) = kpps_hat_t_h;
        end
    end
    Kpps_path_h = Kpps_hat_path_h .* cS.A_path;
end

function PAYG_Contrib_hat_path = aggregate_payg_contributions(dist_path_h, w_path, cS, paramSF)
    % 聚合经济体总的PAYG缴费路径 (hat量)
    nH = cS.nTypes;
    T = cS.T_sim;
    PAYG_Contrib_hat_path = zeros(1, T);
    w_hat_path = w_path ./ cS.A_path;

    for t = 1:T
        total_contrib_t_hat = 0;
        aR_t = cS.aR_new_path(t); % 使用时变退休年龄
        for h = 1:nH
            dist_t_h = dist_path_h{h}(:,:,:,:,t);
            cS_h = cS;
            cS_h.ageEffV_new = cS.ageEffV_new_h(:,h);
            
            % 聚合类型h在t期的劳动供给
            L_h_t = 0;
            for ia = 1:aR_t
                mass_ia = dist_t_h(:,:,:,ia);
                if sum(mass_ia(:)) < 1e-30, continue; end
                le_grid_slice = reshape(paramSF.leGridV, [1, 1, cS.nw_expanded]);
                labor_supply_slice = cS_h.ageEffV_new(ia) .* le_grid_slice;
                L_h_t = L_h_t + sum(labor_supply_slice .* mass_ia, 'all');
            end
            
            % 计算类型h在t期的总缴费
            total_contrib_t_hat = total_contrib_t_hat + cS.theta_path_h(h,t) * w_hat_path(t) * L_h_t;
        end
        PAYG_Contrib_hat_path(t) = total_contrib_t_hat;
    end
end

### 表4.6-0 PAYG主导转向PPS主导的福利损失分析

### 图4.8-1 PAYG主导与PPS主导的路径对比

In [ ]:
% =========================================================================
% == SCRIPT: analyze_pps_structural_reform_welfare.m
% == 版本: [v1.1 - 缓存中间结果版]
% ==
% == 目的:
% ==   1. 比较'structural_reform'与'moderate'两种PPS发展路径下的福利差异。
% ==   2. 通过重构价值函数路径，计算各年龄、各类型家庭的补偿变化(CV)。
% ==   3. 生成LaTeX表格，以揭示结构性改革的福利代价在不同群体间的分配。
% ==
% == v1.1 核心修改:
% ==   - [!!!] 新增了缓存机制。脚本会检查是否存在包含已计算价值函数的
% ==     中间文件。如果存在，则跳过耗时的反向迭代计算，直接加载结果。
% ==   - 如果不存在中间文件，则执行计算并将结果保存，供后续运行使用。
% =========================================================================

clear; close all; clc;
addpath(pwd);
fprintf('=== PPS结构性改革福利分析脚本 (v1.1) ===\n\n');

%% --- 1. 用户设定与文件定义 ---
fprintf('--- 1. 用户设定与文件定义 ---\n');

% --- 分析设定 ---
reform_scenario = 'structural_reform';
base_scenario = 'moderate'; % 福利分析的基准情景

% --- 文件路径模板 ---
file_tpi_pattern = 'TRANS/TPI_results_het_pps_%s.mat';

% --- [核心修改] 中间结果缓存文件 ---
intermediate_results_file = 'results/welfare_analysis_pps_reform_intermediate.mat';

% --- 输出路径 ---
output_dir_tex = 'tex/tab/';
if ~exist(output_dir_tex, 'dir'), mkdir(output_dir_tex); end
output_tex_table_welfare = fullfile(output_dir_tex, 'tab4.6_0_welfare_pps_structural_reform.tex');

fprintf('   改革方案: %s\n', reform_scenario);
fprintf('   基准方案: %s\n', base_scenario);
fprintf('   LaTeX 表格将输出至: %s\n', output_tex_table_welfare);


%% --- 2. [数据加载] 加载所有情景的计算结果 ---
fprintf('\n--- 2. [数据加载] 循环加载方案结果 ---\n');

results_all = struct();
scenarios_to_load = {base_scenario, reform_scenario};

for i = 1:length(scenarios_to_load)
    scenario_name = scenarios_to_load{i};
    fprintf('   正在加载方案: [%s] ...\n', upper(scenario_name));

    file_tpi = sprintf(file_tpi_pattern, scenario_name);
    if ~exist(file_tpi, 'file')
        error('找不到文件: %s，请确认main_run_trans_pps.m已针对该情景运行。', file_tpi); 
    end

    results_all.(scenario_name) = load(file_tpi);
    fprintf('   ✅ 已加载: %s\n', file_tpi);
end
fprintf('--- 所有情景数据加载完毕 ---\n');


%% --- 3. [福利分析] 为各情景重构价值函数路径 (或从缓存加载) ---
fprintf('\n--- 3. [福利分析] 重构价值函数路径 ---\n');

skip_computation = false;
if exist(intermediate_results_file, 'file')
    fprintf('   发现中间结果文件: %s\n', intermediate_results_file);
    try
        loaded_data = load(intermediate_results_file, 'saved_val_paths');
        if isfield(loaded_data.saved_val_paths, base_scenario) && isfield(loaded_data.saved_val_paths, reform_scenario)
            fprintf('   ✅ 文件内容有效，正在加载已保存的价值函数路径...\n');
            results_all.(base_scenario).Val_path_h = loaded_data.saved_val_paths.(base_scenario);
            results_all.(reform_scenario).Val_path_h = loaded_data.saved_val_paths.(reform_scenario);
            skip_computation = true;
        else
            fprintf('   ⚠️ 文件内容不匹配当前分析的方案，将重新计算。\n');
        end
    catch ME
        fprintf('   ❌ 加载失败: %s。将重新计算。\n', ME.message);
    end
else
    fprintf('   未发现中间结果文件，将执行完整计算。\n');
end


if ~skip_computation
    for i_scen = 1:length(scenarios_to_load)
        scen_name = scenarios_to_load{i_scen};
        fprintf('   正在为方案 [%s] 运行反向迭代...\n', upper(scen_name));
        
        data = results_all.(scen_name);
        cS = data.cS;
        paramSF = data.results.ss_data.paramSF;
        valF_h = data.results.ss_data.valF_h;
        polF_h = data.results.ss_data.polF_h;
        iter_results = data.results.iter_results;
        
        Val_path_h = cell(cS.nTypes, 1);
        
        for h = 1:cS.nTypes
            pathS_h = get_pathS_for_backward(iter_results, h, cS, data.ss0);
            cS_h = get_cs_for_type(cS, h);
            [~, Val_path_h{h}] = household.backward_hh(pathS_h, cS_h, paramSF, valF_h{h}, polF_h{h});
        end
        
        results_all.(scen_name).Val_path_h = Val_path_h;
        fprintf('   ✅ 方案 [%s] 价值函数重构完成。\n', upper(scen_name));
    end

    % [核心修改] 保存计算结果到缓存文件
    fprintf('   正在保存中间计算结果至: %s ...\n', intermediate_results_file);
    saved_val_paths = struct();
    saved_val_paths.(base_scenario) = results_all.(base_scenario).Val_path_h;
    saved_val_paths.(reform_scenario) = results_all.(reform_scenario).Val_path_h;
    save(intermediate_results_file, 'saved_val_paths', '-v7.3');
    fprintf('   ✅ 中间结果保存成功。\n');
end

fprintf('--- 所有情景价值函数路径准备完毕 ---\n');


%% --- 4. [生成表格] 计算并生成福利分析(CV)表格 ---
fprintf('\n--- 4. [生成表格] 计算并生成福利分析(CV)表格 ---\n');

% --- 定义分析维度 ---
assessment_years_welfare = [2030, 2040, 2050, 2070, 2100];
type_labels = {'高收入', '中低收入'};
sector_labels = {'A. 城镇职工', 'B. 城乡居民'};

% --- 获取基准情景的数据 ---
data_base = results_all.(base_scenario);
data_reform = results_all.(reform_scenario);
cS_base = data_base.cS;
sigma = cS_base.sigma;
T_sim_base = cS_base.T_sim;

% --- 定义年龄组 ---
age_groups = {
    '青年 (25-29岁)', round((25 - cS_base.age1_orig)/cS_base.time_Step) + 1;
    '中年 (45-49岁)', round((45 - cS_base.age1_orig)/cS_base.time_Step) + 1;
    '临退 (60-64岁)', cS_base.aR_new_path(1);
    '老年 (70-74岁)', round((70 - cS_base.age1_orig)/cS_base.time_Step) + 1;
};

% --- 打开文件并写入LaTeX表头 ---
fileID_welfare = fopen(output_tex_table_welfare, 'w', 'n', 'UTF-8');
fprintf(fileID_welfare, '%% =======================================================\n');
fprintf(fileID_welfare, '%%  此文件由 analyze_pps_structural_reform_welfare.m 自动生成\n');
fprintf(fileID_welfare, '%% =======================================================\n');
fprintf(fileID_welfare, '\\begin{table}[h!]\n');
fprintf(fileID_welfare, '\\centering\n');
fprintf(fileID_welfare, '\\caption{结构性养老金改革的福利影响 (\\%%)}\n');
fprintf(fileID_welfare, '\\label{tab:welfare_impact_pps_reform}\n');
fprintf(fileID_welfare, '\\begin{threeparttable}\n');

header_cols = ['l', repmat('c', 1, length(assessment_years_welfare))];
fprintf(fileID_welfare, '\\begin{tabular}{%s}\n', header_cols);
fprintf(fileID_welfare, '\\toprule\n');

header_line = '家庭类型与年龄组';
for year = assessment_years_welfare
    header_line = [header_line, sprintf(' & %d年', year)];
end
fprintf(fileID_welfare, '%s \\\\\n', header_line);
fprintf(fileID_welfare, '\\midrule\n');

% --- 核心CV计算与写入 ---
for i_sector = 1:length(sector_labels)
    fprintf(fileID_welfare, '\\multicolumn{%d}{l}{\\textbf{%s}} \\\\\n', 1 + length(assessment_years_welfare), sector_labels{i_sector});
    for i_type = 1:length(type_labels)
        h_idx = (i_sector - 1) * 2 + i_type;
        
        fprintf(fileID_welfare, '    \\hspace{1em}%s', type_labels{i_type});
        fprintf(fileID_welfare, repmat(' & ', 1, length(assessment_years_welfare)));
        fprintf(fileID_welfare, ' \\\\\n');

        for i_age = 1:size(age_groups, 1)
            age_label = age_groups{i_age, 1};
            a_idx = age_groups{i_age, 2};
            
            fprintf(fileID_welfare, '       \\hspace{2em}%s', age_label);

            for i_yr = 1:length(assessment_years_welfare)
                year = assessment_years_welfare(i_yr);
                t_idx = round((year - cS_base.start_year) / cS_base.time_Step) + 1;
                
                cv_str = '-';
                if t_idx <= T_sim_base
                    dist_base_slice = data_base.final_Dist_path_h{h_idx}(:, :, :, a_idx, t_idx);
                    val_base_slice  = data_base.Val_path_h{h_idx}(:, :, :, a_idx, t_idx);
                    val_reform_slice= data_reform.Val_path_h{h_idx}(:, :, :, a_idx, t_idx);
                    
                    mass_slice = sum(dist_base_slice, 'all');
                    
                    cv = NaN;
                    if mass_slice > 1e-12
                        EV_base = sum(val_base_slice .* dist_base_slice, 'all') / mass_slice;
                        EV_reform = sum(val_reform_slice .* dist_base_slice, 'all') / mass_slice;
                        
                        if EV_base < -1e-9 && EV_reform < -1e-9
                            cv = (EV_reform / EV_base)^(1 / (1 - sigma)) - 1;
                        end
                    end
                    cv_str = format_cv(cv);
                end
                fprintf(fileID_welfare, ' & %s', cv_str);
            end % year loop
            fprintf(fileID_welfare, ' \\\\\n');
        end % age group loop
    end % type loop
    if i_sector == 1, fprintf(fileID_welfare, '\\midrule\n'); end
end % sector loop

% --- 计算并写入总体平均 ---
fprintf(fileID_welfare, '\\midrule\n');
fprintf(fileID_welfare, '\\multicolumn{%d}{l}{\\textbf{C. 总体平均}} \\\\\n', 1 + length(assessment_years_welfare));
fprintf(fileID_welfare, '    人口加权平均 CV');

for i_yr = 1:length(assessment_years_welfare)
    year = assessment_years_welfare(i_yr);
    t_idx = round((year - cS_base.start_year) / cS_base.time_Step) + 1;
    
    cv_str = '-';
    if t_idx <= T_sim_base
        total_pop_t = sum(cS_base.Z_path_raw(:, t_idx), 'all');
        total_weighted_ev_base = 0;
        total_weighted_ev_reform = 0;
        
        for h_idx = 1:cS_base.nTypes
            for a_idx = 1:cS_base.aD_new
                dist_base_slice = data_base.final_Dist_path_h{h_idx}(:, :, :, a_idx, t_idx);
                val_base_slice  = data_base.Val_path_h{h_idx}(:, :, :, a_idx, t_idx);
                val_reform_slice= data_reform.Val_path_h{h_idx}(:, :, :, a_idx, t_idx);
                
                total_weighted_ev_base   = total_weighted_ev_base   + sum(val_base_slice .* dist_base_slice, 'all');
                total_weighted_ev_reform = total_weighted_ev_reform + sum(val_reform_slice .* dist_base_slice, 'all');
            end
        end
        
        EV_base_total_avg = total_weighted_ev_base / total_pop_t;
        EV_reform_total_avg = total_weighted_ev_reform / total_pop_t;
        
        cv_total = NaN;
        if EV_base_total_avg < -1e-9 && EV_reform_total_avg < -1e-9
             cv_total = (EV_reform_total_avg / EV_base_total_avg)^(1 / (1 - sigma)) - 1;
        end
        cv_str = format_cv(cv_total);
    end
    fprintf(fileID_welfare, ' & %s', cv_str);
end
fprintf(fileID_welfare, ' \\\\\n');


% --- 写入表尾 ---
fprintf(fileID_welfare, '\\bottomrule\n');
fprintf(fileID_welfare, '\\end{tabular}\n');
fprintf(fileID_welfare, '\\begin{tablenotes}[para,flushleft]\n');
fprintf(fileID_welfare, '  \\item 注：表中数值为补偿变化(CV)，衡量“结构性改革”情景相对于“温和改革”情景的福利差异。负值代表福利损失。例如，-0.5\\%%表示该群体愿意放弃其在温和改革情景下剩余生命周期消费的0.5\\%%，以避免进入结构性改革情景。\n');
fprintf(fileID_welfare, '\\end{tablenotes}\n');
fprintf(fileID_welfare, '\\end{threeparttable}\n');
fprintf(fileID_welfare, '\\end{table}\n');
fclose(fileID_welfare);
fprintf('   ✅ 成功生成 LaTeX 表格文件: %s\n', output_tex_table_welfare);

fprintf('\n--- 脚本执行完毕 ---\n');


%% --- 5. [生成图表] 比较两种情景下的宏观经济路径 ---
fprintf('\n--- 5. [生成图表] 比较两种情景下的宏观经济路径 ---\n');

time_axis = data_base.cS.start_year : data_base.cS.time_Step : (data_base.cS.start_year + (data_base.cS.T_sim-1)*data_base.cS.time_Step);
time_mask = (time_axis >= 2023) & (time_axis <= 2100);
time_axis_plot = time_axis(time_mask);


% --- 定义输出路径 ---
output_dir_fig = 'tex/fig/';
if ~exist(output_dir_fig, 'dir'), mkdir(output_dir_fig); end
output_fig_macro_comparison = fullfile(output_dir_fig, 'fig4.8-1_macro_comparison_pps_reforms.png');

% --- 准备绘图数据 ---
scenarios_to_plot = {base_scenario, reform_scenario};
plot_data = struct();

for i = 1:length(scenarios_to_plot)
    scen_name = scenarios_to_plot{i};
    data = results_all.(scen_name);
    
    % 利率 (年化, %)
    plot_data.(scen_name).r_path = ((1 + data.results.r_path).^(1/data.cS.time_Step) - 1) * 100;
    
    % 工资 (level)
    plot_data.(scen_name).w_path = data.results.w_path;

    % K/Y (私人资本/产出)
    plot_data.(scen_name).Kp_to_Y_path = data.results.K_p_path ./ data.results.Y_path;

    % C/Y (消费/产出)
    plot_data.(scen_name).C_to_Y_path = data.results.C_path ./ data.results.Y_path;
end

% --- 绘图风格设定 ---
fig_macro = figure('Name', '宏观经济路径比较：温和改革 vs. 结构性改革');
fig_macro.Units = 'centimeters';
fig_macro.Position = [15 15 14.99 5.5]; % [left bottom width height]

tcl_macro = tiledlayout(1, 3, 'Padding', 'compact', 'TileSpacing', 'compact');


% 灰度风格定义
style_base = {'-', 'Color', [0.5 0.5 0.5], 'LineWidth', 2.5};
style_reform = {'--', 'Color', [0.0 0.0 0.0], 'LineWidth', 2.5};
legend_labels = {'温和改革 (基准)', '结构性改革'};

% --- 子图1: 利率路径 ---
ax1 = nexttile;
hold(ax1, 'on');
plot(ax1, time_axis_plot, plot_data.(base_scenario).r_path(time_mask), style_base{:});
plot(ax1, time_axis_plot, plot_data.(reform_scenario).r_path(time_mask), style_reform{:});
hold(ax1, 'off');
grid(ax1, 'on'); box(ax1, 'on');
xlim(ax1, [2023, 2100]);
title(ax1, '(a) 年化真实利率(%)', 'FontSize', 10);
xlabel(ax1, '年份');
l1 = legend(ax1, legend_labels, 'position',[0.0765 0.1992 0.1684 0.1562],'IconColumnWidth',10,'Box','off');

% --- 子图2: 工资路径 ---
ax2 = nexttile;
hold(ax2, 'on');
plot(ax2, time_axis_plot, plot_data.(base_scenario).w_path(time_mask), style_base{:});
plot(ax2, time_axis_plot, plot_data.(reform_scenario).w_path(time_mask), style_reform{:});
hold(ax2, 'off');
grid(ax2, 'on'); box(ax2, 'on');
xlim(ax2, [2023, 2100]);
title(ax2, '(b) 工资水平', 'FontSize', 10);
xlabel(ax2, '年份');
l2 = legend(ax2, legend_labels, 'position',[0.3862 0.7083 0.1684 0.1562],'IconColumnWidth',10,'Box','off');

% --- 子图3: 宏观比率 (K/Y 和 C/Y) ---
ax3 = nexttile;
hold(ax3, 'on');
% 绘制 K/Y
p1 = plot(ax3, time_axis_plot, plot_data.(base_scenario).Kp_to_Y_path(time_mask), '-', 'Color', [0.5 0.5 0.5], 'LineWidth', 2.5);
p2 = plot(ax3, time_axis_plot, plot_data.(reform_scenario).Kp_to_Y_path(time_mask), '--', 'Color', [0.0 0.0 0.0], 'LineWidth', 2.5);
% 绘制 C/Y
p3 = plot(ax3, time_axis_plot, plot_data.(base_scenario).C_to_Y_path(time_mask), '-.', 'Color', [0.5 0.5 0.5], 'LineWidth', 2);
p4 = plot(ax3, time_axis_plot, plot_data.(reform_scenario).C_to_Y_path(time_mask), ':', 'Color', [0.0 0.0 0.0], 'LineWidth', 2);
hold(ax3, 'off');
grid(ax3, 'on'); box(ax3, 'on');
xlim(ax3, [2023, 2100]);
title(ax3, '(c) 宏观比率', 'FontSize', 10);
xlabel(ax3, '年份');
l3 = legend([p1 p2 p3 p4], {'K/Y (温和)', 'K/Y (结构性)', 'C/Y (温和)', 'C/Y (结构性)'}, 'position',[0.7013 0.5861 0.1455 0.2957],'IconColumnWidth',10,'Box','off');

%% --- 6. 保存新生成的图表 ---
fprintf('\n--- 6. 正在保存宏观比较图表 ---\n');
try
    exportgraphics(fig_macro, output_fig_macro_comparison, 'Resolution', 300);
    fprintf('   ✅ 成功保存图表至: %s\n', output_fig_macro_comparison);
catch ME
    fprintf('   ❌ 保存失败: %s\n', ME.message);
end

fprintf('\n--- 脚本执行完毕 ---\n');

%% --- [本地辅助函数] ---

function pathS_h = get_pathS_for_backward(iter_results, h, cS, ss0)
    % 为指定的家庭类型 h，打包后向迭代所需的所有时间路径
    pathS_h = struct();
    pathS_h.r_path = iter_results.r_path_iter;
    w_hat_path_converged = iter_results.w_hat_path_iter;
    pathS_h.w_hat_path = w_hat_path_converged;
    pathS_h.tr_per_hh_hat_path = iter_results.TR_hat_pc_path_iter;
    
    w_hat_path_for_b = [ss0.w_hat, w_hat_path_converged(1:end-1)];
    b_hat_formula_path_h_all = zeros(cS.nTypes, cS.T_sim);
    for t = 1:cS.T_sim
        b_hat_formula_path_h_all(:, t) = SS.calculate_formula_benefits(w_hat_path_for_b(t), cS);
    end
    adj_path_converged = iter_results.adj_path_iter;
    pathS_h.b_hat_path = adj_path_converged .* b_hat_formula_path_h_all(h, :);
    
    A_path_ext = [cS.A_path, cS.A_path(end) * (1 + ((1+cS.g_A_ss)^cS.time_Step-1))];
    g_A_path = A_path_ext(2:end)./A_path_ext(1:end-1)-1;
    pathS_h.g_A_path = g_A_path(1:cS.T_sim);
    pathS_h.theta_path = cS.theta_path_h(h, :);
end

function cS_h = get_cs_for_type(cS, h)
    % 为指定的家庭类型 h，准备一个专属的 cS 参数包
    cS_h = cS;
    cS_h.ageEffV_new = cS.ageEffV_new_h(:, h);
end

function str_out = format_cv(cv_value)
    % 辅助函数: 将CV数值格式化为带正负号和百分号的字符串
    if isnan(cv_value)
        str_out = '-';
        return;
    end
    
    if cv_value >= 0
        sign_char = '+';
    else
        sign_char = ''; % 负号会自动显示
    end
    str_out = sprintf('%s%.2f\\%%', sign_char, cv_value * 100);
end

### 表4.6 不同个人养老金(PPS)改革方案对退休人员综合福利替代水平的影响

### 附录--表1.1-引入个人养老金账户(PPS)的福利影响 (ambitious方案 vs. 无PPS情景, 补偿变化 CV, \%)

### 附录--表1.1-引入个人养老金账户(PPS)的福利影响 (conservative方案 vs. 无PPS情景, 补偿变化 CV, \%)

### 附录--表1.1-引入个人养老金账户(PPS)的福利影响 (moderate方案 vs. 无PPS情景, 补偿变化 CV, \%)

In [ ]:
% =========================================================================
% == SCRIPT: tab_pps.m
% == 版本: [v1.4 - 引入缓存机制]
% ==
% == 核心任务:
% ==   1. [新增] 引入缓存机制，将耗时计算结果保存，避免重复运行。
% ==   2. [重构] 将计算部分与图表生成部分分离。
% ==   3. [保留] 生成 表 4.5(现图4.8)、表 4.6 及附录福利表。
% =========================================================================

clear; close all; clc;
addpath(pwd);
fprintf('=== PPS改革情景对比分析脚本 (v1.4) ===\n\n');

%% --- 0. [!!!] 快速测试模式开关 ---
quick_test_mode = false;
if quick_test_mode
    fprintf('!!! 警告: 快速测试模式已开启。结果将不被缓存，且附录福利分析结果将不准确。!!!\n');
end
% =====================================================================

%% --- 1. 用户设定与文件定义 ---
fprintf('\n--- 1. 用户设定与文件定义 ---\n');

scenarios = {'None', 'conservative', 'moderate', 'ambitious'};
base_scenario_welfare = 'None';
scenarios_for_welfare_comp = {'conservative', 'moderate', 'ambitious'};

file_tpi_pattern = 'TRANS/PPS/TPI_results_het_pps_%s.mat';
file_tpi_nopps = 'TRANS/PPS/TPI_results_het_nopps.mat';

% --- 输出路径定义 ---
output_dir_tex = 'tex/tab/';
output_dir_fig = 'tex/fig/';
output_dir_results = 'results/'; % [新增] 缓存文件目录
if ~exist(output_dir_tex, 'dir'), mkdir(output_dir_tex); end
if ~exist(output_dir_fig, 'dir'), mkdir(output_dir_fig); end
if ~exist(output_dir_results, 'dir'), mkdir(output_dir_results); end

% --- 输出文件名定义 ---
output_tex_table_macro = fullfile(output_dir_tex, 'tab4.5_macro_pps.tex'); % 注意：此脚本不再生成此表
output_tex_table_adj = fullfile(output_dir_tex, 'tab4.6_adj_sustainability_pps.tex');
appendix_file_pattern = 'Appendix_tab1.1_welfare_analysis_pps_%s.tex';
output_fig_pps = fullfile(output_dir_fig, 'fig4.8_macro_pps.png');

% --- [新增] 缓存文件定义 ---
output_file_cache = fullfile(output_dir_results, 'tab_pps_processed_data.mat');
fprintf('   LaTeX 表格将输出至: %s\n', output_dir_tex);
fprintf('   图像将输出至: %s\n', output_dir_fig);
fprintf('   计算结果缓存文件为: %s\n', output_file_cache);


%% --- 2. [核心逻辑] 加载或计算处理后数据 ---
% 检查缓存文件是否存在。如果存在且不是快速测试模式，则加载；否则，执行完整计算。
if exist(output_file_cache, 'file') && ~quick_test_mode
    fprintf('\n--- 2. [缓存加载] 发现已处理数据, 直接加载... ---\n');
    load(output_file_cache, 'results_all', 'plot_data');
    fprintf('   ✅ 已加载: %s\n', output_file_cache);
else
    if quick_test_mode
        fprintf('\n--- 2. [重新计算] 快速测试模式开启, 将强制重新计算 (结果不保存) ---\n');
    else
        fprintf('\n--- 2. [重新计算] 未发现缓存文件, 开始执行完整计算... ---\n');
    end

    % --- 2.1. [数据加载] 加载所有情景的计算结果 ---
    fprintf('\n--- 2.1. [数据加载] 循环加载各方案结果 ---\n');
    results_all = struct();
    for i = 1:length(scenarios)
        scenario_name = scenarios{i};
        fprintf('   正在加载方案: [%s] ...\n', upper(scenario_name));
        if strcmp(scenario_name, 'None'), file_tpi = file_tpi_nopps;
        else, file_tpi = sprintf(file_tpi_pattern, scenario_name); end
        if ~exist(file_tpi, 'file'), error('找不到文件: %s', file_tpi); end
        results_all.(scenario_name) = load(file_tpi);
        fprintf('   ✅ 已加载: %s\n', file_tpi);
    end
    fprintf('--- 所有情景数据加载完毕 ---\n');

    % --- 2.2. [福利分析预处理] 为所有相关情景重构价值函数路径 ---
    fprintf('\n--- 2.2. [福利分析预处理] 重构价值函数路径 ---\n');
    scenarios_needing_vfi = [{base_scenario_welfare}, scenarios_for_welfare_comp];
    for i = 1:length(scenarios_needing_vfi)
        scen_name = scenarios_needing_vfi{i};
        fprintf('   正在为方案 [%s] 准备价值函数路径...\n', upper(scen_name));
        data = results_all.(scen_name);
        cS = data.cS;
        paramSF = data.results.ss_data.paramSF;
        valF_h = data.results.ss_data.valF_h;
        polF_h = data.results.ss_data.polF_h;
        iter_results = data.results.iter_results;

        if quick_test_mode
            fprintf('      >>> [快速模式] 跳过VFI, 创建占位符Val_path_h...\n');
            T = cS.T_sim;
            Val_path_h_placeholder = cell(cS.nTypes, 1);
            for h = 1:cS.nTypes
                Val_path_h_placeholder{h} = repmat(valF_h{h}, [1, 1, 1, 1, T]);
            end
            results_all.(scen_name).Val_path_h = Val_path_h_placeholder;
        else
            fprintf('      >>> [完整模式] 启动VFI反向迭代 (此过程可能较慢)...\n');
            Val_path_h = cell(cS.nTypes, 1);
            parfor h = 1:cS.nTypes % 可以考虑使用parfor加速
                pathS_h = get_pathS_for_backward(iter_results, h, cS, data.ss0);
                cS_h = get_cs_for_type(cS, h);
                [~, Val_path_h{h}] = household.backward_hh(pathS_h, cS_h, paramSF, valF_h{h}, polF_h{h});
            end
            results_all.(scen_name).Val_path_h = Val_path_h;
            fprintf('      >>> [完整模式] VFI反向迭代完成。\n');
        end
    end
    fprintf('--- 所有情景价值函数路径准备完毕 ---\n');
    
    % --- 2.3. [数据计算] 计算绘图所需数据 ---
    fprintf('\n--- 2.3. [数据计算] 为图4.8准备数据 ---\n');
    plot_data = struct();
    scenarios_for_fig_calc = {'conservative', 'moderate', 'ambitious'};
    
    % -- 计算恒定的PAYG/GDP比率 (只需计算一次) --
    data_ref = results_all.None;
    cS_ref = data_ref.cS;
    paramSF_ref = data_ref.results.ss_data.paramSF;
    t_idx_ref = 5; % 任选一个时间点
    aR_t_ref = cS_ref.aR_new_path(t_idx_ref);
    w_hat_t_ref = data_ref.results.iter_results.w_hat_path_iter(t_idx_ref);
    total_payg_contrib_ref = 0;
    for h = 1:cS_ref.nTypes
        L_h_t = TPI.get_labor_supply_path_for_type(h, {data_ref.final_Dist_path_h{h}(:,:,:,:,t_idx_ref)}, cS_ref, paramSF_ref, aR_t_ref);
        total_payg_contrib_ref = total_payg_contrib_ref + cS_ref.theta_path_h(h,t_idx_ref) * w_hat_t_ref * L_h_t;
    end
    Y_hat_t_ref = data_ref.results.Y_path(t_idx_ref) / cS_ref.A_path(t_idx_ref);
    plot_data.payg_gdp_ratio = total_payg_contrib_ref / Y_hat_t_ref;

    % -- 循环计算各PPS情景的PPS/GDP比率路径 --
    for i_scen = 1:length(scenarios_for_fig_calc)
        scen_name = scenarios_for_fig_calc{i_scen};
        data = results_all.(scen_name);
        cS = data.cS;
        paramSF = data.results.ss_data.paramSF;
        T = cS.T_sim;
        pps_gdp_ratio_path = zeros(1, T);

        for t = 1:T
            Y_hat_t = data.results.Y_path(t) / cS.A_path(t);
            aR_t = cS.aR_new_path(t);
            w_hat_t = data.results.iter_results.w_hat_path_iter(t);
            total_pps_contrib_t = 0;

            for h = 1:cS.nTypes
                cS_h = get_cs_for_type(cS, h);
                pol_t_h = data.final_Pol_path_h{h}{t};
                dist_t_h = data.final_Dist_path_h{h}(:,:,:,:,t);
                for ia = 1:aR_t
                    mass_ia_h = dist_t_h(:,:,:,ia);
                    if sum(mass_ia_h,'all') < 1e-30, continue; end
                    le_grid_slice = reshape(paramSF.leGridV, [1,1,cS.nw_expanded]);
                    labor_income_grid = w_hat_t * cS_h.ageEffV_new(ia) .* le_grid_slice;
                    labor_income_grid_full = repmat(labor_income_grid, [cS.nk, cS.nkpps, 1]);
                    pps_rate_grid = pol_t_h(ia).pps_contrib_rate;
                    total_pps_contrib_t = total_pps_contrib_t + sum(pps_rate_grid .* labor_income_grid_full .* mass_ia_h, 'all');
                end
            end

            if abs(Y_hat_t) > 1e-9, pps_gdp_ratio_path(t) = total_pps_contrib_t / Y_hat_t; end
        end
        plot_data.(scen_name).pps_gdp_ratio = pps_gdp_ratio_path;
    end
    fprintf('   ✅ 绘图数据计算完成。\n');

    % --- 2.4. [adj分析预处理] 计算各情景PPS给付总额 ---
    fprintf('\n--- 2.4. [adj分析预处理] 计算各情景PPS给付总额 ---\n');
    for i = 1:length(scenarios)
        scen_name = scenarios{i};
        data = results_all.(scen_name);
        cS = data.cS;
        T = cS.T_sim;
        total_pps_withdrawal_hat_path = zeros(1, T);
        if ~cS.pps_active
            results_all.(scen_name).total_pps_withdrawal_hat_path = total_pps_withdrawal_hat_path;
            fprintf('   方案 [%s] 无PPS, 给付额为0。\n', upper(scen_name));
            continue;
        end
        fprintf('   正在为方案 [%s] 计算PPS给付路径...\n', upper(scen_name));
        kpps_grid_vec = cS.kppsGridV;
        kpps_mat = repmat(reshape(kpps_grid_vec, [1, cS.nkpps]), [cS.nk, 1, cS.nw_expanded]);
        r_path = data.results.iter_results.r_path_iter;
        for t = 1:T
            aR_t = cS.aR_new_path(t);
            withdrawal_rate_t = cS.pps_withdrawal_rate;
            for h = 1:cS.nTypes
                dist_t_h = data.final_Dist_path_h{h}(:,:,:,:,t);
                for ia = (aR_t + 1) : cS.aD_new
                    dist_slice = dist_t_h(:, :, :, ia);
                    if sum(dist_slice, 'all') < 1e-30, continue; end
                    pps_asset_return_hat = kpps_mat * (1 + r_path(t));
                    pps_withdrawal_grid = withdrawal_rate_t * pps_asset_return_hat;
                    total_pps_withdrawal_hat_path(t) = total_pps_withdrawal_hat_path(t) + sum(pps_withdrawal_grid .* dist_slice, 'all');
                end
            end
        end
        results_all.(scen_name).total_pps_withdrawal_hat_path = total_pps_withdrawal_hat_path;
    end
    fprintf('--- 所有情景PPS给付路径计算完毕 ---\n');
    
    % --- 2.5. [保存缓存] ---
    if ~quick_test_mode
        fprintf('\n--- 2.5. [保存缓存] 所有计算完成, 保存结果到文件... ---\n');
        save(output_file_cache, 'results_all', 'plot_data', '-v7.3');
        fprintf('   ✅ 结果已保存至: %s\n', output_file_cache);
    end
end
fprintf('--- 数据准备完毕 ---\n');


%% --- 3. [生成图4.8] PPS与PAYG缴费占GDP比重路径 ---
fprintf('\n--- 3. [生成图4.8] PPS与PAYG缴费占GDP比重路径 ---\n');

% --- 1. 定义绘图参数 ---
scenarios_for_fig = {'conservative', 'moderate', 'ambitious'};
color_map = containers.Map(...
    {'conservative', 'moderate', 'ambitious'}, ...
    {[0.65 0.65 0.65], [0.0 0.0 0.0], [0.35 0.35 0.35]}); % 灰度: 浅, 黑, 深
line_style_map = containers.Map(...
    {'conservative', 'moderate', 'ambitious'}, ...
    {':', '-', '--'});
legend_label_map = containers.Map(...
    {'conservative', 'moderate', 'ambitious'}, ...
    {'保守方案', '中性方案', '激进方案'});

% --- 2. 绘图 ---
cS = results_all.moderate.cS; % 使用任一情景获取时间轴
T = cS.T_sim;
time_axis = cS.start_year : cS.time_Step : (cS.start_year + (T-1)*cS.time_Step);

fig_pps_gdp = figure('Name', 'PPS and PAYG Contribution Rates', 'Position', [100 428 425 172]);
hold on;

% -- 绘制恒定的PAYG比率虚线 --
yline(plot_data.payg_gdp_ratio * 100, ...
    'Color', [0.5 0.5 0.5], ...
    'LineStyle', ':', ...
    'LineWidth', 2, ...
    'DisplayName', 'PAYG缴费率');

% -- 循环绘制各PPS情景路径 --
for i = 1:length(scenarios_for_fig)
    scen_name = scenarios_for_fig{i};
    plot(time_axis, plot_data.(scen_name).pps_gdp_ratio * 100, ...
        'Color', color_map(scen_name), ...
        'LineStyle', line_style_map(scen_name), ...
        'LineWidth', 2.5, ...
        'DisplayName', legend_label_map(scen_name));
end

% --- 3. 添加图表格式 ---
xlabel('年份', 'FontName', 'SimSun');
ylabel('占GDP比重 (%)', 'FontName', 'SimSun');
legend('show', 'Location', 'best', 'FontName', 'SimSun');
grid on;
box on;
xlim([2023, 2100]);

% --- 4. 保存图像 ---
fprintf('   正在保存缴费率路径图至: %s\n', output_fig_pps);
try
    exportgraphics(fig_pps_gdp, output_fig_pps, 'Resolution', 300);
    fprintf('   ✅ 图像保存成功。\n');
catch ME
    fprintf('   ❌ 图像保存失败！错误信息: %s\n', ME.message);
end


%% --- 4. [生成附录表] 循环生成各情景福利分析(CV)表格 ---
fprintf('\n--- 4. [生成附录表] 循环生成各情景福利分析(CV)表格 ---\n');

% [!!! 核心修正: 在此逻辑块内明确定义所需变量 !!!]
assessment_years_welfare = [2023, 2030, 2040, 2050, 2070, 2100];
type_labels = {'高收入', '中低收入'};
sector_labels = {'A. 城镇职工', 'B. 城乡居民'};

data_base = results_all.(base_scenario_welfare);
cS_base = data_base.cS;
sigma = cS_base.sigma;
T_sim_base = cS_base.T_sim;

age_groups = {
    '青年 (25-29岁)', round((25 - cS_base.age1_orig)/cS_base.time_Step) + 1;
    '中年 (45-49岁)', round((45 - cS_base.age1_orig)/cS_base.time_Step) + 1;
    '临退 (60-64岁)', cS_base.aR_new_path(1);
    '老年 (70-74岁)', round((70 - cS_base.age1_orig)/cS_base.time_Step) + 1;
};

for i_reform_scen = 1:length(scenarios_for_welfare_comp)
    reform_scenario_name = scenarios_for_welfare_comp{i_reform_scen};
    data_reform = results_all.(reform_scenario_name);

    output_tex_table_welfare = fullfile(output_dir_tex, sprintf(appendix_file_pattern, reform_scenario_name));
    fprintf('   正在生成福利表: %s\n', output_tex_table_welfare);

    fileID_welfare = fopen(output_tex_table_welfare, 'w', 'n', 'UTF-8');
    fprintf(fileID_welfare, '%% =======================================================\n');
    fprintf(fileID_welfare, '%%  此文件由 tab_pps.m (v1.4) 自动生成\n');
    fprintf(fileID_welfare, '%% =======================================================\n');
    fprintf(fileID_welfare, '\\begin{table}[h!]\n');
    fprintf(fileID_welfare, '\\centering\n');
    fprintf(fileID_welfare, '\\caption{引入个人养老金账户(PPS)的福利影响 (%s方案 vs. 无PPS情景, 补偿变化 CV, \\%%)}\n', reform_scenario_name);
    fprintf(fileID_welfare, '\\label{tab:welfare_impact_pps_%s}\n', reform_scenario_name);
    fprintf(fileID_welfare,'\\small\n');
    fprintf(fileID_welfare, '\\begin{threeparttable}\n');

    header_cols = ['l', repmat('c', 1, length(assessment_years_welfare))];
    header_line_1 = strjoin(cellfun(@(x) sprintf('%d', x), num2cell(assessment_years_welfare), 'UniformOutput', false), ' & ');
    fprintf(fileID_welfare, '\\begin{tabular}{%s}\n', header_cols);
    fprintf(fileID_welfare, '\\toprule\n');
    fprintf(fileID_welfare, '家庭类型与年龄组 & %s \\\\\n', header_line_1);
    fprintf(fileID_welfare, '\\midrule\n');

    for i_sector = 1:length(sector_labels)
        fprintf(fileID_welfare, '\\multicolumn{%d}{l}{\\textbf{%s}} \\\\\n', length(assessment_years_welfare) + 1, sector_labels{i_sector});
        for i_type = 1:length(type_labels)
            h_idx = (i_sector - 1) * 2 + i_type;
            fprintf(fileID_welfare, '    \\hspace{1em}%s', type_labels{i_type});
            fprintf(fileID_welfare, repmat(' & ', 1, length(assessment_years_welfare)));
            fprintf(fileID_welfare, ' \\\\\n');
            for i_age = 1:size(age_groups, 1)
                age_label = age_groups{i_age, 1};
                a_idx = age_groups{i_age, 2};
                fprintf(fileID_welfare, '       \\hspace{2em}%s', age_label);
                for i_yr = 1:length(assessment_years_welfare)
                    year = assessment_years_welfare(i_yr);
                    t_idx = round((year - cS_base.start_year) / cS_base.time_Step) + 1;
                    cv = NaN;
                    if t_idx > 0 && t_idx <= T_sim_base
                        dist_base_slice = data_base.final_Dist_path_h{h_idx}(:,:,:,a_idx,t_idx);
                        val_base_slice  = data_base.Val_path_h{h_idx}(:,:,:,a_idx,t_idx);
                        val_reform_slice= data_reform.Val_path_h{h_idx}(:,1,:,a_idx,t_idx);
                        mass_slice = sum(dist_base_slice, 'all');
                        if mass_slice > 1e-12
                            EV_base = sum(val_base_slice .* dist_base_slice, 'all') / mass_slice;
                            EV_reform = sum(val_reform_slice .* dist_base_slice, 'all') / mass_slice;
                            if EV_base < -1e-9 && EV_reform < -1e-9
                                cv = (EV_reform / EV_base)^(1 / (1 - sigma)) - 1;
                            end
                        end
                    end
                    fprintf(fileID_welfare, ' & %s', format_cv(cv));
                end
                fprintf(fileID_welfare, ' \\\\\n');
            end
        end
    end

    fprintf(fileID_welfare, '\\midrule\n');
    fprintf(fileID_welfare, '\\textbf{总体平均}');
    for i_yr = 1:length(assessment_years_welfare)
        year = assessment_years_welfare(i_yr);
        t_idx = round((year - cS_base.start_year) / cS_base.time_Step) + 1;
        cv_total = NaN;
        if t_idx > 0 && t_idx <= T_sim_base
            total_pop_t = sum(cS_base.Z_path_raw(:, t_idx));
            total_weighted_ev_base_all = 0;
            total_weighted_ev_reform_all = 0;
            for h_idx = 1:cS_base.nTypes
                for a_idx = 1:cS_base.aD_new
                    dist_base_slice = data_base.final_Dist_path_h{h_idx}(:,:,:,a_idx,t_idx);
                    val_base_slice  = data_base.Val_path_h{h_idx}(:,:,:,a_idx,t_idx);
                    val_reform_slice= data_reform.Val_path_h{h_idx}(:,1,:,a_idx,t_idx);
                    total_weighted_ev_base_all = total_weighted_ev_base_all + sum(val_base_slice .* dist_base_slice, 'all');
                    total_weighted_ev_reform_all = total_weighted_ev_reform_all + sum(val_reform_slice .* dist_base_slice, 'all');
                end
            end
            if total_pop_t > 1e-9
                EV_base_total_avg = total_weighted_ev_base_all / total_pop_t;
                EV_reform_total_avg = total_weighted_ev_reform_all / total_pop_t;
                if EV_base_total_avg < -1e-9 && EV_reform_total_avg < -1e-9
                     cv_total = (EV_reform_total_avg / EV_base_total_avg)^(1 / (1 - sigma)) - 1;
                end
            end
        end
        fprintf(fileID_welfare, ' & %s', format_cv(cv_total));
    end
    fprintf(fileID_welfare, ' \\\\\n');

    fprintf(fileID_welfare, '\\bottomrule\n');
    fprintf(fileID_welfare, '\\end{tabular}\n');
    fprintf(fileID_welfare, '\\begin{tablenotes}[para,flushleft]\n');
    fprintf(fileID_welfare, '  \\item 注：表中数值为补偿变化(CV)，衡量引入个人养老金账户相对于无个人养老金账户情景的福利差异。正值代表福利改善。\n');
    fprintf(fileID_welfare, '\\end{tablenotes}\n');
    fprintf(fileID_welfare, '\\end{threeparttable}\n');
    fprintf(fileID_welfare, '\\end{table}\n');
    fclose(fileID_welfare);
    fprintf('   ✅ 成功生成 LaTeX 表格文件: %s\n', output_tex_table_welfare);
end


%% --- 5. [生成表4.6] 计算并生成adj对比表格 ---
fprintf('\n--- 5. [生成表4.6] 计算并生成adj对比表格 ---\n');
assessment_years_adj = [2030, 2040, 2050, 2060, 2070, 2100];
scenarios_for_adj = {'None', 'conservative', 'moderate', 'ambitious'};
scenario_labels_adj = {'无PPS', '保守方案', '中性方案', '激进方案'};
fileID_adj = fopen(output_tex_table_adj, 'w', 'n', 'UTF-8');
fprintf(fileID_adj, '%% =======================================================\n');
fprintf(fileID_adj, '%%  此文件由 tab_pps.m (v1.4) 自动生成\n');
fprintf(fileID_adj, '%% =======================================================\n');
fprintf(fileID_adj, '\\begin{table}[h!]\n');
fprintf(fileID_adj, '\\centering\n');
fprintf(fileID_adj, '\\caption{不同个人养老金改革方案对退休人员综合福利替代水平的影响}\n');
fprintf(fileID_adj, '\\label{tab4.6:adj_welfare_pps}\n');
fprintf(fileID_adj, '\\small\n');
fprintf(fileID_adj, '\\begin{threeparttable}\n');

header_cols = 'l';
header_line_1 = {'年份'};
header_line_2_parts = {''};
cmidrule_parts = {};
current_col_idx = 1;
for i = 1:length(scenarios_for_adj)
    scen_label = scenario_labels_adj{i};
    if strcmp(scenarios_for_adj{i}, 'None')
        header_cols = [header_cols, 'c'];
        header_line_1{end+1} = sprintf('%s', scen_label);
        header_line_2_parts{end+1} = 'PAYG';
        cmidrule_parts{end+1} = sprintf('\\cmidrule(lr){%d-%d}', current_col_idx + 1, current_col_idx + 1);
        current_col_idx = current_col_idx + 1;
    else
        header_cols = [header_cols, 'rrr'];
        header_line_1{end+1} = sprintf('\\multicolumn{3}{c}{%s}', scen_label);
        header_line_2_parts = [header_line_2_parts, {'PAYG', '综合', '缴费率'}];
        cmidrule_parts{end+1} = sprintf('\\cmidrule(lr){%d-%d}', current_col_idx + 1, current_col_idx + 3);
        current_col_idx = current_col_idx + 3;
    end
end
final_header_1 = strjoin(header_line_1, ' & ');
final_header_2 = strjoin(header_line_2_parts, ' & ');
final_cmidrule = strjoin(cmidrule_parts, ' ');

fprintf(fileID_adj, '\\begin{tabular}{%s}\n', header_cols);
fprintf(fileID_adj, '\\toprule\n');
fprintf(fileID_adj, '%s \\\\\n', final_header_1);
fprintf(fileID_adj, '%s\n', final_cmidrule);
fprintf(fileID_adj, '%s \\\\\n', final_header_2);
fprintf(fileID_adj, '\\midrule\n');

for i_yr = 1:length(assessment_years_adj)
    year = assessment_years_adj(i_yr);
    fprintf(fileID_adj, '%d', year);
    for i_scen = 1:length(scenarios_for_adj)
        scen_name = scenarios_for_adj{i_scen};
        data = results_all.(scen_name);
        cS = data.cS;
        paramSF = data.results.ss_data.paramSF;
        t_idx = round((year - cS.start_year) / cS.time_Step) + 1;
        adj_payg = NaN;
        adj_comp = NaN;
        avg_pps_rate = NaN;
        if t_idx > 0 && t_idx <= cS.T_sim
            adj_payg = data.results.iter_results.adj_path_iter(t_idx);
            w_hat_for_b = data.ss0.w_hat;
            if t_idx > 1, w_hat_for_b = data.results.iter_results.w_hat_path_iter(t_idx-1); end
            aR_t = cS.aR_new_path(t_idx);
            b_hat_formula_h = SS.calculate_formula_benefits(w_hat_for_b, cS);
            total_formula_benefit_t = 0;
            for h = 1:cS.nTypes
                mass_retirees_h_t = sum(cS.Z_path_raw((aR_t + 1):end, t_idx)) * cS.type_weights(h);
                total_formula_benefit_t = total_formula_benefit_t + b_hat_formula_h(h) * mass_retirees_h_t;
            end
            if abs(total_formula_benefit_t) > 1e-9
                actual_payg_benefit_t = adj_payg * total_formula_benefit_t;
                pps_withdrawal_t = data.total_pps_withdrawal_hat_path(t_idx);
                adj_comp = (actual_payg_benefit_t + pps_withdrawal_t) / total_formula_benefit_t;
            end
            if ~strcmp(scen_name, 'None')
                w_hat_t = data.results.iter_results.w_hat_path_iter(t_idx);
                total_pps_contrib_t = 0;
                total_labor_income_workers_t = 0;
                for h = 1:cS.nTypes
                    cS_h = get_cs_for_type(cS, h);
                    dist_t_h = data.final_Dist_path_h{h}(:,:,:,:,t_idx);
                    pol_t_h = data.final_Pol_path_h{h}{t_idx};
                    for ia = 1:aR_t
                        mass_ia_h = dist_t_h(:,:,:,ia);
                        if sum(mass_ia_h, 'all') < 1e-30, continue; end
                        le_grid_slice = reshape(paramSF.leGridV, [1, 1, cS.nw_expanded]);
                        labor_income_grid = w_hat_t * cS_h.ageEffV_new(ia) .* le_grid_slice;
                        labor_income_grid_full = repmat(labor_income_grid, [cS.nk, cS.nkpps, 1]);
                        pps_rate_grid = pol_t_h(ia).pps_contrib_rate;
                        total_pps_contrib_t = total_pps_contrib_t + sum(pps_rate_grid .* labor_income_grid_full .* mass_ia_h, 'all');
                        total_labor_income_workers_t = total_labor_income_workers_t + sum(labor_income_grid_full .* mass_ia_h, 'all');
                    end
                end
                if abs(total_labor_income_workers_t) > 1e-9
                    avg_pps_rate = total_pps_contrib_t / total_labor_income_workers_t;
                else, avg_pps_rate = 0; end
            end
        end
        if strcmp(scen_name, 'None')
            fprintf(fileID_adj, ' & %.3f', adj_payg);
        else
            fprintf(fileID_adj, ' & %.3f & %.3f & %s', adj_payg, adj_comp, format_rate(avg_pps_rate));
        end
    end
    fprintf(fileID_adj, ' \\\\\n');
end

fprintf(fileID_adj, '\\bottomrule\n');
fprintf(fileID_adj, '\\end{tabular}\n');
fprintf(fileID_adj, '\\begin{tablenotes}[para,flushleft]\n');
fprintf(fileID_adj, '  \\item 注： “PAYG”列为在各情景下，为保证PAYG体系自身收支平衡所需的福利调整因子。“综合”列为一个模拟的综合福利兑现因子，其计算方式为 (实际PAYG福利 + PPS给付) / 公式化PAYG福利，衡量了引入PPS后，退休人员总养老金收入相对于原定PAYG福利水平的提升程度。“缴费率”为该年份所有工作年龄人口的个人养老金缴费总额占其劳动收入总额的加权平均比率。\n');
fprintf(fileID_adj, '\\end{tablenotes}\n');
fprintf(fileID_adj, '\\end{threeparttable}\n');
fprintf(fileID_adj, '\\end{table}\n');
fclose(fileID_adj);
fprintf('   ✅ 成功生成 LaTeX 表格文件: %s\n', output_tex_table_adj);

fprintf('\n--- 脚本执行完毕 ---\n');



%% --- [本地辅助函数] ---

function pathS_h = get_pathS_for_backward(iter_results, h, cS, ss0)
    % =====================================================================
    % == 函数: get_pathS_for_backward (本地辅助函数)
    % == 目的: 为指定的家庭类型 h，打包后向迭代所需的所有时间路径。
    % =====================================================================
    pathS_h = struct();
    pathS_h.r_path = iter_results.r_path_iter;
    w_hat_path_converged = iter_results.w_hat_path_iter;
    pathS_h.w_hat_path = w_hat_path_converged;
    pathS_h.tr_per_hh_hat_path = iter_results.TR_hat_pc_path_iter;

    w_hat_path_for_b = [ss0.w_hat, w_hat_path_converged(1:end-1)];
    b_hat_formula_path_h_all = zeros(cS.nTypes, cS.T_sim);
    for t = 1:cS.T_sim
        b_hat_formula_path_h_all(:, t) = SS.calculate_formula_benefits(w_hat_path_for_b(t), cS);
    end
    adj_path_converged = iter_results.adj_path_iter;
    pathS_h.b_hat_path = adj_path_converged .* b_hat_formula_path_h_all(h, :);

    A_path_ext = [cS.A_path, cS.A_path(end) * (1 + ((1+cS.g_A_ss)^cS.time_Step-1))];
    g_A_path = A_path_ext(2:end)./A_path_ext(1:end-1)-1;
    pathS_h.g_A_path = g_A_path(1:cS.T_sim);
    pathS_h.theta_path = cS.theta_path_h(h, :);
end

function cS_h = get_cs_for_type(cS, h)
    % =====================================================================
    % == 函数: get_cs_for_type (本地辅助函数)
    % == 版本: 1.1 - 修正PPS路径索引错误
    % == 目的: 为指定的家庭类型 h，准备一个专属的 cS 参数包。
    % =====================================================================
    cS_h = cS;
    cS_h.ageEffV_new = cS.ageEffV_new_h(:, h);
end

function str_out = format_cv(cv_value)
    % 辅助函数: 将CV数值格式化为带正负号和百分号的字符串
    if isnan(cv_value)
        str_out = '--';
        return;
    end

    if cv_value >= 0
        sign_char = '+';
    else
        sign_char = '';
    end
    str_out = sprintf('%s%.2f\\%%', sign_char, cv_value * 100);
end

function str_out = format_rate(rate_value)
    % 辅助函数: 将缴费率格式化为百分比字符串
    if isnan(rate_value)
        str_out = '--';
    else
        str_out = sprintf('%.2f\\%%', rate_value * 100);
    end
end

## 弹性劳动供给

### 表 4.7a--弹性劳动供给模型下高收入家庭的平均劳动供给决策

### 表 4.7b--不同劳动供给模型下的关键宏观变量对比

In [ ]:
% =========================================================================
% == SCRIPT: tab_flexiblelabor.m
% == 版本: [v1.4 - 表格拆分版]
% ==
% == 目的:
% ==   1. 生成两个独立的LaTeX表格:
% ==      - tex/tab/tab4.7_a.tex (劳动供给决策)
% ==      - tex/tab/tab4.7_b.tex (宏观变量对比)
% ==   2. 保持v1.3版本的所有计算逻辑和格式优化。
% =========================================================================

% --- 1. 初始化与设定 ---
clear; close all; clc;
addpath(pwd);
fprintf('=== 弹性/固定劳动供给对比分析脚本 (v1.4) ===\n\n');

% --- 分析设定 ---
scenarios = {'flexible', 'fixed'};

% --- [核心修改] 路径设定: 定义两个输出文件 ---
file_tpi_pattern = 'v1_flexible_labor/TRANS/TPI_results_het_nopps_%s.mat';
output_dir_tex = 'tex/tab/';
if ~exist(output_dir_tex, 'dir'), mkdir(output_dir_tex); end
output_tex_table_a = 'tex/tab/tab4.7_a_flexible_labor.tex';
output_tex_table_b = 'tex/tab/tab4.7_b_flexible_marco.tex';

% --- 表格参数 ---
assessment_years = [2030, 2040, 2050, 2060, 2070, 2100];

type_labels = {'高收入', '中低收入'};
sector_labels = {'A. 城镇职工', 'B. 城乡居民'};

macro_indicators = {'y_hat_pc', 'k_p_hat_pc', 'c_hat_pc', 'r', 'adj'};
macro_indicator_labels = {'有效人均产出 ($\hat{y}$)', '有效人均私人资本 ($\hat{k}_p$)', ...
                           '有效人均消费 ($\hat{c}$)', ...
                          '年化真实利率 (r, \%)', '养老金均衡调整因子 (adj)'};


%% --- 2. [数据加载] 加载所有情景的计算结果 ---
fprintf('--- 2. [数据加载] 循环加载各方案结果 ---\n');
results_all = struct();
for i = 1:length(scenarios)
    scenario_name = scenarios{i};
    fprintf('   正在加载方案: [%s] ...\n', upper(scenario_name));
    file_tpi = sprintf(file_tpi_pattern, scenario_name);
    if ~exist(file_tpi, 'file')
        error('找不到文件: %s，请确认main_run_trans.m已针对该情景运行。', file_tpi); 
    end
    results_all.(scenario_name) = load(file_tpi);
    fprintf('   ✅ 已加载: %s\n', file_tpi);
end
fprintf('--- 所有情景数据加载完毕 ---\n');


%% --- 3. [Panel A 计算] 平均劳动供给 ---
fprintf('\n--- 3. [Panel A 计算] 平均劳动供给决策 (l_choice) ---\n');
data_flex = results_all.flexible;
cS_flex = data_flex.cS;
paramS_flex = data_flex.results.ss_data.paramSF; 

age_groups = {
    '青年 (25-29岁)', round((25 - cS_flex.age1_orig)/cS_flex.time_Step) + 1;
    '中年 (45-49岁)', round((45 - cS_flex.age1_orig)/cS_flex.time_Step) + 1;
    '临退 (60-64岁)', cS_flex.aR_new_path(1); 
};

panel_a_data = nan(length(sector_labels) * length(type_labels) * size(age_groups, 1), length(assessment_years));

leGridV = paramS_flex.leGridV; 
[~, ~, le_grid_full] = ndgrid(cS_flex.kGridV, cS_flex.kppsGridV, leGridV); 

row_idx_a = 1;
for i_sector = 1:length(sector_labels)
    for i_type = 1:length(type_labels)
        h_idx = (i_sector - 1) * 2 + i_type;
        for i_age = 1:size(age_groups, 1)
            a_idx = age_groups{i_age, 2};
            
            age_eff_ia_h = cS_flex.ageEffV_new_h(a_idx, h_idx);
            if age_eff_ia_h < 1e-9, row_idx_a = row_idx_a + 1; continue; end
            
            denominator_grid = age_eff_ia_h .* le_grid_full;
            denominator_grid(abs(denominator_grid) < 1e-9) = 1e-9;

            for i_yr = 1:length(assessment_years)
                year = assessment_years(i_yr);
                t_idx = round((year - cS_flex.start_year) / cS_flex.time_Step) + 1;
                
                if t_idx > cS_flex.T_sim, continue; end
                
                dist_slice = data_flex.final_Dist_path_h{h_idx}(:, :, :, a_idx, t_idx);
                l_effective_slice = data_flex.final_Pol_path_h{h_idx}{t_idx}(a_idx).l;
                
                l_choice_slice = l_effective_slice ./ denominator_grid;
                
                mass_total = sum(dist_slice, 'all');
                if mass_total > 1e-12
                    weighted_l_choice = sum(dist_slice .* l_choice_slice, 'all');
                    avg_l_choice = weighted_l_choice / mass_total;
                    panel_a_data(row_idx_a, i_yr) = avg_l_choice;
                end
            end
            row_idx_a = row_idx_a + 1;
        end
    end
end
fprintf('   ✅ Panel A (平均劳动决策) 数据计算完成。\n');


%% --- 4. [Panel B 计算] 宏观变量(有效人均)对比 ---
fprintf('\n--- 4. [Panel B 计算] 宏观变量(有效人均)对比 ---\n');
panel_b_data = struct();

for i_scen = 1:length(scenarios)
    scen_name = scenarios{i_scen};
    data = results_all.(scen_name);
    results = data.results;
    cS = data.cS;
    
    total_pop_path = sum(cS.Z_path_raw, 1);
    Y_hat_pc_path = (results.Y_path ./ cS.A_path) ./ total_pop_path;
    K_p_hat_pc_path = (results.K_p_path ./ cS.A_path) ./ total_pop_path;
    C_hat_pc_path = (results.C_path ./ cS.A_path) ./ total_pop_path;
    
    aggr_supply = aggregates.get_path_aggregates(data.final_Dist_path_h, data.final_Pol_path_h, cS, results.ss_data.paramSF);
    L_hat_pc_path = aggr_supply.L_path ./ total_pop_path;

    r_path = results.iter_results.r_path_iter;
    adj_path = results.iter_results.adj_path_iter;
    
    paths = containers.Map(macro_indicators, {Y_hat_pc_path, K_p_hat_pc_path, C_hat_pc_path, r_path, adj_path});
    
    for i_ind = 1:length(macro_indicators)
        indicator = macro_indicators{i_ind};
        current_path = paths(indicator);
        for i_yr = 1:length(assessment_years)
            year = assessment_years(i_yr);
            t_idx = round((year - cS.start_year) / cS.time_Step) + 1;
            
            if t_idx > cS.T_sim, value = NaN;
            else
                value = current_path(t_idx);
                if strcmp(indicator, 'r'), value = ((1 + value)^(1/cS.time_Step) - 1) * 100; end
            end
            panel_b_data.(scen_name).(indicator)(i_yr) = value;
        end
    end
end
fprintf('   ✅ Panel B 数据计算完成。\n');


%% --- 5. [生成 LaTeX 表格] ---
fprintf('\n--- 5. [生成 LaTeX 表格] ---\n');

% --- 5a. 生成表格 a: 劳动供给决策 ---
fileID_a = fopen(output_tex_table_a, 'w', 'n', 'UTF-8');
fprintf(fileID_a, '%% =======================================================\n');
fprintf(fileID_a, '%%  此文件由 tab_flexiblelabor.m (v1.4) 自动生成\n');
fprintf(fileID_a, '%% =======================================================\n');
fprintf(fileID_a, '\\begin{table}[h!]\n');
fprintf(fileID_a, '\\centering\n');
fprintf(fileID_a, '\\caption{弹性劳动供给模型下高收入家庭的平均劳动供给决策}\n');
fprintf(fileID_a, '\\label{tab4.7a:labor_decision}\n');
fprintf(fileID_a, '\\begin{threeparttable}\n');
fprintf(fileID_a, '\\begin{tabular}{lcccccc}\n');
fprintf(fileID_a, '\\toprule\n');
fprintf(fileID_a, '家庭类型与年龄组');
for year = assessment_years, fprintf(fileID_a, ' & %d', year); end
fprintf(fileID_a, ' \\\\\n');
fprintf(fileID_a, '\\midrule\n');

row_idx_a = 1;
for i_sector = 1:length(sector_labels)
    fprintf(fileID_a, '\\multicolumn{7}{l}{\\textbf{%s}} \\\\\n', sector_labels{i_sector});
    i_type = 1; % 只打印高收入组
    % for i_type = 1:length(type_labels)
    fprintf(fileID_a, '    \\hspace{1em}%s', type_labels{i_type});
    fprintf(fileID_a, repmat(' & ', 1, length(assessment_years)));
    fprintf(fileID_a, ' \\\\\n');
    for i_age = 1:size(age_groups, 1)
        fprintf(fileID_a, '        \\hspace{2em}%s', age_groups{i_age, 1});
        for i_yr = 1:length(assessment_years)
            val = panel_a_data(row_idx_a, i_yr);
            if isnan(val), fprintf(fileID_a, ' & -'); else, fprintf(fileID_a, ' & %.3f', val); end
        end
        fprintf(fileID_a, ' \\\\\n');
        row_idx_a = row_idx_a + 1;
    end
    % end
    row_idx_a = row_idx_a + size(age_groups, 1); % 跳过低收入组的数据行索引
    if i_sector < length(sector_labels), fprintf(fileID_a, '\\addlinespace\n'); end
    
end

fprintf(fileID_a, '\\bottomrule\n');
fprintf(fileID_a, '\\end{tabular}\n');
fprintf(fileID_a, '\\begin{tablenotes}[para,flushleft]\n');
fprintf(fileID_a, '  \\item 注： 表中数值为特定工作年龄群体将其时间禀赋用于市场工作的平均比例。为表格简洁，仅展示高收入群体决策；所有情景下，中低收入群体的平均劳动决策均为1.0。所有情景均基于无个人养老金账户(nopps)的设定。\n');
fprintf(fileID_a, '\\end{tablenotes}\n');
fprintf(fileID_a, '\\end{threeparttable}\n');
fprintf(fileID_a, '\\end{table}\n');
fclose(fileID_a);
fprintf('   ✅ 成功生成 LaTeX 表格文件: %s\n', output_tex_table_a);


% --- 5b. 生成表格 b: 宏观变量对比 ---
fileID_b = fopen(output_tex_table_b, 'w', 'n', 'UTF-8');
fprintf(fileID_b, '%% =======================================================\n');
fprintf(fileID_b, '%%  此文件由 tab_flexiblelabor.m (v1.4) 自动生成\n');
fprintf(fileID_b, '%% =======================================================\n');
fprintf(fileID_b, '\\begin{table}[h!]\n');
fprintf(fileID_b, '\\centering\n');
fprintf(fileID_b, '\\caption{不同劳动供给模型下的关键宏观变量对比 (有效人均量)}\n');
fprintf(fileID_b, '\\label{tab4.7b:macro_comparison}\n');
fprintf(fileID_b, '\\begin{threeparttable}\n');
fprintf(fileID_b, '\\begin{tabular}{lcccccc}\n');
fprintf(fileID_b, '\\toprule\n');
fprintf(fileID_b, '宏观指标');
for year = assessment_years, fprintf(fileID_b, ' & %d', year); end
fprintf(fileID_b, ' \\\\\n');
fprintf(fileID_b, '\\midrule\n');

for i_ind = 1:length(macro_indicators)
    indicator = macro_indicators{i_ind};
    fprintf(fileID_b, '%s', macro_indicator_labels{i_ind});
    fprintf(fileID_b, repmat(' & ', 1, length(assessment_years)));
    fprintf(fileID_b, ' \\\\\n');
    
    for i_scen = 1:length(scenarios)
        scen_name = scenarios{i_scen};
        if strcmp(scen_name, 'flexible'), row_label = '弹性供给'; else, row_label = '固定供给'; end
        fprintf(fileID_b, '    \\hspace{1em}%s', row_label);
        
        for i_yr = 1:length(assessment_years)
            val = panel_b_data.(scen_name).(indicator)(i_yr);
             if isnan(val), fprintf(fileID_b, ' & -'); else, fprintf(fileID_b, ' & %.3f', val); end
        end
        fprintf(fileID_b, ' \\\\\n');
    end
    if i_ind < length(macro_indicators), fprintf(fileID_b, '\\addlinespace\n'); end
end

fprintf(fileID_b, '\\bottomrule\n');
fprintf(fileID_b, '\\end{tabular}\n');
fprintf(fileID_b, '\\begin{tablenotes}[para,flushleft]\n');
fprintf(fileID_b, '  \\item 注： 表中对比了两种模型下的关键宏观变量路径，所有总量均为剔除技术(A)和人口(N)双重影响后的有效人均量。固定供给模型假设所有工作年龄个体均提供1单位的劳动。所有情景均基于无个人养老金账户(nopps)的设定。\n');
fprintf(fileID_b, '\\end{tablenotes}\n');
fprintf(fileID_b, '\\end{threeparttable}\n');
fprintf(fileID_b, '\\end{table}\n');
fclose(fileID_b);
fprintf('   ✅ 成功生成 LaTeX 表格文件: %s\n', output_tex_table_b);

fprintf('\n--- 脚本执行完毕 ---\n');

## 随机效率冲击

### 表 4.8 --随机劳动效率冲击与无冲击模型的关键宏观变量对比

In [ ]:


% --- 1. 初始化与设定 ---
clear; close all; clc;
addpath(pwd);
fprintf('=== 弹性/固定劳动供给对比分析脚本 (v1.4) ===\n\n');

% --- 分析设定 ---
scenarios = {'incomeshock', 'raw'};

% --- [核心修改] 路径设定: 定义两个输出文件 ---
file_tpi_pattern = 'TRANS/TPI_results_het_nopps_%s.mat';
output_dir_tex = 'tex/tab/';
if ~exist(output_dir_tex, 'dir'), mkdir(output_dir_tex); end
output_tex_table_b = 'tex/tab/tab4.8_incomeshock_marco.tex';

% --- 表格参数 ---
assessment_years = [2030, 2040, 2050, 2060, 2070, 2100];

type_labels = {'高收入', '中低收入'};
sector_labels = {'A. 城镇职工', 'B. 城乡居民'};

macro_indicators = {'y_hat_pc', 'k_p_hat_pc', 'c_hat_pc', 'r', 'adj'};
macro_indicator_labels = {'有效人均产出 ($\hat{y}$)', '有效人均私人资本 ($\hat{k}_p$)', ...
                           '有效人均消费 ($\hat{c}$)', ...
                          '年化真实利率 (r, \%)', '养老金均衡调整因子 (adj)'};


%% --- 2. [数据加载] 加载所有情景的计算结果 ---
fprintf('--- 2. [数据加载] 循环加载各方案结果 ---\n');
results_all = struct();
for i = 1:length(scenarios)
    scenario_name = scenarios{i};
    fprintf('   正在加载方案: [%s] ...\n', upper(scenario_name));
    file_tpi = sprintf(file_tpi_pattern, scenario_name);
    if ~exist(file_tpi, 'file')
        error('找不到文件: %s，请确认main_run_trans.m已针对该情景运行。', file_tpi); 
    end
    results_all.(scenario_name) = load(file_tpi);
    fprintf('   ✅ 已加载: %s\n', file_tpi);
end
fprintf('--- 所有情景数据加载完毕 ---\n');



%% --- 4. [Panel B 计算] 宏观变量(有效人均)对比 ---
fprintf('\n--- 4. [Panel B 计算] 宏观变量(有效人均)对比 ---\n');
panel_b_data = struct();

for i_scen = 1:length(scenarios)
    scen_name = scenarios{i_scen};
    data = results_all.(scen_name);
    results = data.results;
    cS = data.cS;
    
    total_pop_path = sum(cS.Z_path_raw, 1);
    Y_hat_pc_path = (results.Y_path ./ cS.A_path) ./ total_pop_path;
    K_p_hat_pc_path = (results.K_p_path ./ cS.A_path) ./ total_pop_path;
    C_hat_pc_path = (results.C_path ./ cS.A_path) ./ total_pop_path;
    
    aggr_supply = aggregates.get_path_aggregates(data.final_Dist_path_h, data.final_Pol_path_h, cS, results.ss_data.paramSF);
    L_hat_pc_path = aggr_supply.L_path ./ total_pop_path;

    r_path = results.iter_results.r_path_iter;
    adj_path = results.iter_results.adj_path_iter;
    
    paths = containers.Map(macro_indicators, {Y_hat_pc_path, K_p_hat_pc_path, C_hat_pc_path, r_path, adj_path});
    
    for i_ind = 1:length(macro_indicators)
        indicator = macro_indicators{i_ind};
        current_path = paths(indicator);
        for i_yr = 1:length(assessment_years)
            year = assessment_years(i_yr);
            t_idx = round((year - cS.start_year) / cS.time_Step) + 1;
            
            if t_idx > cS.T_sim, value = NaN;
            else
                value = current_path(t_idx);
                if strcmp(indicator, 'r'), value = ((1 + value)^(1/cS.time_Step) - 1) * 100; end
            end
            panel_b_data.(scen_name).(indicator)(i_yr) = value;
        end
    end
end
fprintf('   ✅ Panel B 数据计算完成。\n');


%% --- 5. [生成 LaTeX 表格] ---

% --- 5b. 生成表格 b: 宏观变量对比 ---
fileID_b = fopen(output_tex_table_b, 'w', 'n', 'UTF-8');
fprintf(fileID_b, '%% =======================================================\n');
fprintf(fileID_b, '%%  此文件由 tab_flexiblelabor.m (v1.4) 自动生成\n');
fprintf(fileID_b, '%% =======================================================\n');
fprintf(fileID_b, '\\begin{table}[h!]\n');
fprintf(fileID_b, '\\centering\n');
fprintf(fileID_b, '\\caption{随机劳动效率冲击与无冲击模型的关键宏观变量对比(有效人均量)}\n');
fprintf(fileID_b, '\\label{tab4.8:incomeshock_macro_comparison}\n');
fprintf(fileID_b, '\\begin{threeparttable}\n');
fprintf(fileID_b, '\\begin{tabular}{lcccccc}\n');
fprintf(fileID_b, '\\toprule\n');
fprintf(fileID_b, '宏观指标');
for year = assessment_years, fprintf(fileID_b, ' & %d', year); end
fprintf(fileID_b, ' \\\\\n');
fprintf(fileID_b, '\\midrule\n');

for i_ind = 1:length(macro_indicators)
    indicator = macro_indicators{i_ind};
    fprintf(fileID_b, '%s', macro_indicator_labels{i_ind});
    fprintf(fileID_b, repmat(' & ', 1, length(assessment_years)));
    fprintf(fileID_b, ' \\\\\n');
    
    for i_scen = 1:length(scenarios)
        scen_name = scenarios{i_scen};
        if strcmp(scen_name, 'incomeshock'), row_label = '随机冲击'; else, row_label = '无冲击'; end
        fprintf(fileID_b, '    \\hspace{1em}%s', row_label);
        
        for i_yr = 1:length(assessment_years)
            val = panel_b_data.(scen_name).(indicator)(i_yr);
             if isnan(val), fprintf(fileID_b, ' & -'); else, fprintf(fileID_b, ' & %.3f', val); end
        end
        fprintf(fileID_b, ' \\\\\n');
    end
    if i_ind < length(macro_indicators), fprintf(fileID_b, '\\addlinespace\n'); end
end

fprintf(fileID_b, '\\bottomrule\n');
fprintf(fileID_b, '\\end{tabular}\n');
fprintf(fileID_b, '\\begin{tablenotes}[para,flushleft]\n');
fprintf(fileID_b, '  \\item 注： 表中对比了两种模型下的关键宏观变量路径，所有总量均为剔除技术(A)和人口(N)双重影响后的有效人均量。所有情景均基于无个人养老金账户(nopps)的设定。\n');
fprintf(fileID_b, '\\end{tablenotes}\n');
fprintf(fileID_b, '\\end{threeparttable}\n');
fprintf(fileID_b, '\\end{table}\n');
fclose(fileID_b);
fprintf('   ✅ 成功生成 LaTeX 表格文件: %s\n', output_tex_table_b);

fprintf('\n--- 脚本执行完毕 ---\n');